In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import fnmatch
import numpy as np
import pickle
import matplotlib.pyplot as plt
import umap
from sklearn.mixture import GaussianMixture
from scipy import stats
# from sklearn.cluster import OPTICS
from copy import deepcopy

import pandas as pd

# import tensorflow as tf
# from tensorflow.keras import layers
# from tensorflow.keras.backend import mean
# from tensorflow.keras.backend import square
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import CuDNNLSTM
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import RepeatVector
# from tensorflow.keras.layers import TimeDistributed
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.layers import Flatten

# from tensorflow.keras.utils import Sequence
# from tensorflow.keras import Input
# from tensorflow.keras import Model
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import Conv1D
from scipy.stats import zscore

plt.rcParams['figure.figsize'] = (5.0, 5.0)
plt.rcParams.update({'font.size': 12})
plt.rcParams.update(plt.rcParamsDefault)

np.random.seed(seed=11)


cwd = os.getcwd()

if cwd.split("/")[1] == "export":
    data_dir = "../../../files_from_snuffy"
else:
    data_dir = "../../../data_GRS1915"


In [ ]:
with open('{}/1776_light_curves_1s_bin_errorfix.pkl'.format(data_dir), 'rb') as f:
    lcs = pickle.load(f)
with open('{}/1776_light_curves_1s_bin_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    ids = pickle.load(f)

In [ ]:
# load light curve segments
with open('{}/468202_len128_s2_4cad_counts_errorfix.pkl'.format(data_dir), 'rb') as f:
    segments_counts = pickle.load(f)
    
# load latent variables for light curve segments
weights_dir = "../../../model_weights/model_2020-08-30_11-42-38.h5"
segments_dir = '../../../data_GRS1915/468202_len128_s2_4cad_counts_errorfix.pkl'
segment_encoding_dir = '{}/segment_encoding_{}_segments_{}.pkl'.format(data_dir, weights_dir.split("/")[-1].split(".")[0], segments_dir.split("/")[-1].split(".")[0])
with open(segment_encoding_dir, 'rb') as f:
    segment_encoding = pickle.load(f)

# take latent variable means, i.e. 16 values per segment
segment_encoding_scaled_means = zscore(segment_encoding[:,0,:], axis=0).astype(np.float32)  # standardize per feature

# calculate statistical moments for the segments
desc_stats = np.zeros((len(segments_counts), 4)) #mean, std, skew, kurt
desc_stats[:,0] = np.mean(segments_counts, axis=1).flatten()
desc_stats[:,1] = np.std(segments_counts, axis=1).flatten()
desc_stats[:,2] = stats.skew(segments_counts, axis=1).flatten()
desc_stats[:,3] = stats.kurtosis(segments_counts, axis=1).flatten()
zscore_desc_stats = zscore(desc_stats, axis=0)

# merge the two types of features; shape of shape_moments is [468202, 20]
shape_moments = np.hstack((segment_encoding_scaled_means, zscore_desc_stats)) # every column is standardized

In [ ]:
# fit a number of models for 10k subsets of segments, and for a range of numbers of component
# this was run over a couple of attempts, but using this exact loop, 
# so I put together the bic values and plotted the together in the next cell
# [10,25,50,70,80,90,100,110,120,130,140]
# [110,115,120,125,130,135,140]#50k
component_no_list = np.concatenate(([10, 25, 50, 70, 80, 90, 100], np.arange(110,142, 2), [150]))
subset_size = 50e3
no_subsets = 5
data_subsets = [(int(start*subset_size), int((1+start)*subset_size)) for start in range(no_subsets)][3:]
# gaussian_mixture_bics = np.zeros((len(data_subsets),len(component_no_list)))

gaussian_mixture_bics= np.loadtxt("gaussian_mixture_bics_new_model_10-150_50k.csv", delimiter=",")

for set_iteration, data_set in enumerate(data_subsets):
    for comp_iteration, component_no in enumerate(component_no_list):
        if gaussian_mixture_bics[set_iteration,comp_iteration] != 0.:
            continue
        GMmodel = GaussianMixture(n_components=component_no, random_state=0)
        GMmodel.fit(shape_moments[data_set[0]:data_set[1],:])
        gaussian_mixture_bics[set_iteration,comp_iteration] = GMmodel.bic(shape_moments)

        np.savetxt('gaussian_mixture_bics_new_model_10-150_50k.csv', gaussian_mixture_bics, delimiter =', ') 
        print(set_iteration, comp_iteration)
        
# component_no_list = [150]
# subset_size = 50e3
# no_subsets = 3
# data_subsets = [(int(start*subset_size), int((1+start)*subset_size)) for start in range(no_subsets)]
# gaussian_mixture_bics = np.zeros((len(data_subsets),len(component_no_list)))

# for set_iteration, data_set in enumerate(data_subsets):
#     for comp_iteration, component_no in enumerate(component_no_list):
#         if gaussian_mixture_bics[set_iteration,comp_iteration] != 0.:
#             continue
#         GMmodel = GaussianMixture(n_components=component_no, random_state=0)
#         GMmodel.fit(shape_moments[data_set[0]:data_set[1],:])
#         gaussian_mixture_bics[set_iteration,comp_iteration] = GMmodel.bic(shape_moments)

#         np.savetxt('gaussian_mixture_bics_new_model_150comps_50k.csv', gaussian_mixture_bics, delimiter =', ') 
#         print(set_iteration, comp_iteration)
# #         clear_output(wait=True)
# # index 8 did not converge

In [ ]:
# fit a couple extra models at 160 and 170 components
component_no_list = [180,190, 200]
subset_size = 50e3
no_subsets = 5
data_subsets = [(int(start*subset_size), int((1+start)*subset_size)) for start in range(no_subsets)]
gaussian_mixture_bics = np.zeros((len(data_subsets),len(component_no_list)))

# gaussian_mixture_bics= np.loadtxt("gaussian_mixture_bics_new_model_10-150_50k.csv", delimiter=",")

for set_iteration, data_set in enumerate(data_subsets):
    for comp_iteration, component_no in enumerate(component_no_list):
        if gaussian_mixture_bics[set_iteration,comp_iteration] != 0.:
            continue
        GMmodel = GaussianMixture(n_components=component_no, random_state=0)
        GMmodel.fit(shape_moments[data_set[0]:data_set[1],:])
        gaussian_mixture_bics[set_iteration,comp_iteration] = GMmodel.bic(shape_moments)

        np.savetxt('gaussian_mixture_bics_160-170_50k.csv', gaussian_mixture_bics, delimiter =', ') 
        print(set_iteration, comp_iteration)
        
# component_no_list = [150]
# subset_size = 50e3
# no_subsets = 3
# data_subsets = [(int(start*subset_size), int((1+start)*subset_size)) for start in range(no_subsets)]
# gaussian_mixture_bics = np.zeros((len(data_subsets),len(component_no_list)))

# for set_iteration, data_set in enumerate(data_subsets):
#     for comp_iteration, component_no in enumerate(component_no_list):
#         if gaussian_mixture_bics[set_iteration,comp_iteration] != 0.:
#             continue
#         GMmodel = GaussianMixture(n_components=component_no, random_state=0)
#         GMmodel.fit(shape_moments[data_set[0]:data_set[1],:])
#         gaussian_mixture_bics[set_iteration,comp_iteration] = GMmodel.bic(shape_moments)

#         np.savetxt('gaussian_mixture_bics_new_model_150comps_50k.csv', gaussian_mixture_bics, delimiter =', ') 
#         print(set_iteration, comp_iteration)
# #         clear_output(wait=True)
# # index 8 did not converge

In [ ]:
# gaussian_mixture_bics= np.loadtxt("gaussian_mixture_bics_new_model_10-150_50k.csv", delimiter=",")

gaussian_mixture_bics

In [ ]:
bics = [1.853893282449822500e+07, 1.722017647717545927e+07, 1.658319324933229759e+07, 1.630327464005833492e+07, 1.622106282425292768e+07, 1.616687566882342100e+07, 1.615730210213817097e+07, 1.613055099964568764e+07, 1.611427361123152077e+07, 1.614833229564627074e+07, 1.612438706897565909e+07, 1.610314177309925482e+07, 1.611599805476822704e+07, 1.611190407219867036e+07, 1.613187932035312243e+07, 1.612686166295694374e+07, 1.610543674698158540e+07, 1.610464430899482593e+07, 1.612001376647870429e+07, 1.613567941551903449e+07, 1.613709874137604795e+07, 1.613797303043186106e+07, 1.613711653342354484e+07]
comps = np.concatenate(([10, 25, 50, 70, 80, 90, 100], np.arange(110,142, 2)))

In [ ]:
# data_subsets = [(0,50000), (50000, 100000), (100000,150000),(150000,200000), (200000, 250000)]
# component_no_list = [10,25,50,70,80,90,100,105,110,111,112,113,114,115,116,117,118,119,120,125,130,140,150,200,300]
# bics = np.array([
# [1.513076963327877596e+07, 1.373494201573342085e+07, 1.310223591745524108e+07, 1.287644645175125077e+07, 1.274916968428172357e+07, 1.275498408091743104e+07, 1.272725532548042759e+07,1.273327056713533960e+07, 1.273110944201433845e+07, 1.271215291174531914e+07, 1.269974317490880564e+07, 1.269413576213483885e+07, 1.269831422006917745e+07, 1.270286540602960251e+07, 1.272652081320444494e+07, 1.268854252120269835e+07, 1.271878362980804965e+07, 1.268805628370952979e+07, 1.268268194913305156e+07,1.269743282206927985e+07, 1.269253136956013553e+07, 1.269973950701179914e+07, 1.271462149632845633e+07, 1.292783153119732440e+07, 1.339544358913458139e+07],
# [1.517668787782970630e+07, 1.375655116087884270e+07, 1.301894537214316800e+07, 1.278502388570801169e+07, 1.274546161574119143e+07, 1.272605305849449150e+07, 1.265987061649004370e+07,1.273297575760359690e+07, 1.264832313142326102e+07, 1.267118953481068090e+07, 1.264825245518371463e+07, 1.269095693081272021e+07, 1.266793768580078520e+07, 1.268845881720614433e+07, 1.266137312288295850e+07, 1.268985047919468395e+07, 1.271996471582287923e+07, 1.273072799564955197e+07, 1.269101905685216561e+07,1.270770922392238863e+07, 1.270597545123521052e+07, 1.268562237603535876e+07, 1.274607550342624076e+07, 1.291487403943038359e+07, 1.345206031202976964e+07],
# [1.513147027530102991e+07, 1.372693726637074538e+07, 1.312636607615234889e+07, 1.280763613113106042e+07, 1.276878601520187221e+07, 1.274302309312643856e+07, 1.271846136874406226e+07,1.267622275037734210e+07, 1.266478125283669680e+07, 1.272365127627760172e+07, 1.270341859422520176e+07, 1.274606540429916233e+07, 1.270035337163357809e+07, 1.269462134574233554e+07, 1.264591281519959494e+07, 1.266692204580633529e+07, 1.268642471454811655e+07, 1.267109306289681979e+07, 1.267652125702439621e+07,1.268893210948343575e+07, 1.268379260471562855e+07, 1.266906201551120169e+07, 1.271607317416701652e+07, 1.287800719977548718e+07, 1.346352189637094550e+07],
# [1.513075834188815393e+07, 1.372975220068028197e+07, 1.305765112890582718e+07, 1.283192171104326844e+07, 1.274878631334129721e+07, 1.271947265357426368e+07, 1.270828195666158944e+07,1.271268768232382834e+07, 1.263517875820080563e+07, 1.269493846218724176e+07, 1.264023021297656186e+07, 1.266014644823007658e+07, 1.266488662979799323e+07, 1.264187690211539157e+07, 1.264225268117410690e+07, 1.263162761737927422e+07, 1.262308756661838479e+07, 1.260968587468877248e+07, 1.261931922283876687e+07,1.265345469640533440e+07, 1.265581449083781987e+07, 1.267555561351045780e+07, 1.271492636046844162e+07, 1.291254769987491146e+07, 1.341979996590973809e+07],
# [1.509176813460635953e+07, 1.379692069009907730e+07, 1.309409092464722134e+07, 1.283283915952128172e+07, 1.278231797994311526e+07, 1.271217661442268454e+07, 1.264713496262365766e+07,1.270169793278488889e+07, 1.265528527531241067e+07, 1.265825935773974657e+07, 1.268149589179967158e+07, 1.265726369034977630e+07, 1.265107941581576318e+07, 1.266242553794561140e+07, 1.266178248237481155e+07, 1.267541880952490866e+07, 1.269726119435581937e+07, 1.270745073380917870e+07, 1.265447378898119554e+07,1.270840458273771219e+07, 1.268563932986871898e+07, 1.268250343190071173e+07, 1.269332119142067619e+07, 1.287152213201368600e+07, 1.345943013665321097e+07]
# ])
bics = gaussian_mixture_bics[:2]
minimums = []
for iteration, dataset_bics in enumerate(bics):
    minimums.append(component_no_list[np.argmin(dataset_bics)])
    plt.plot(component_no_list,dataset_bics, label=data_subsets[iteration])
print("Minimum BIC values: ", minimums)
print("Mean of minimum BIC values: ", np.mean(minimums))
print("Minimum of mean BIC values (minimum of cyan): ",component_no_list[np.argmin(np.mean(bics, axis=0))])
plt.plot(component_no_list,np.mean(bics, axis=0), marker = "o", c="cyan", label="mean")
plt.xlim([90,140])
plt.ylim([1.61e7, 1.62e7])
plt.legend()
plt.title("BIC values for Gaussian mixture models fit to 5 data subsets of 50k segments")
plt.xlabel("number of Gaussian components")
plt.ylabel("Bayesian information criterion")
plt.show()

In [ ]:
GMmodel = GaussianMixture(n_components=128, random_state=0,verbose=2)
GMmodel.fit(shape_moments)
GMmodel_dir = "{}/GMM_128comp_20d_alldata_model_2020-08-30_11-42-38.pkl".format(data_dir)
with open(GMmodel_dir, 'wb') as f:
    pickle.dump(GMmodel, f)
    
# copy of the segment encoding in /export/data/jakubok/GRS1915+105>
# pickled GMM is named segment_encoding_model_2020-08-30_11-42-38_segments_468202_len128_s2_4cad_counts_errorfix.pkl

In [ ]:
16696+6*12000

In [ ]:

# shape_moments_GMM122_labels = GMmodel.predict(shape_moments)
# with open("{}/shape_moments_GMM122_labels".format(data_dir), 'wb') as f:
#     pickle.dump(shape_moments_GMM122_labels, f)

# Comparison 122 components Gaussian Mixture Model vs Belloni classifications

In [ ]:
with open("{}/shape_moments_GMM122_labels.pkl".format(data_dir), 'rb') as f: # output of LSTM autoencoder's decoder
    shape_moments_GMM122_labels = pickle.load(f)

In [ ]:
# load observation classifications from Huppenkothen 2017
clean_belloni = open('{}/1915Belloniclass_updated.dat'.format(data_dir))
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
    if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state
        
# make a dict of Tomaso's classifications  (Daniela's set) against observation ids,
# i.e. ob_state == {'20187-02-01-00': 'alpha', '20187-02-01-01': 'alpha', '20402-01-22-00': 'alpha', ...}
# lines = clean_belloni.readlines()
# states = lines[0].split()
# belloni_clean = {}
# for h,l in zip(states, lines[1:]):
#     belloni_clean[h] = l.split()
#     #state: obsID1, obsID2...
# ob_state = {}
# for state, obs in belloni_clean.items():
#     if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
#     for ob in obs:
#         ob_state[ob] = state
        
        
# load IDs of segmented light curves: observationsID_segmentIndex
with open('{}/468202_len128_s2_4cad_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    seg_ids = pickle.load(f)

        
seg_ObIDs = [seg.split("_")[0] for seg in seg_ids] # get rid of the within-observation segment indices and create a degenerate list of observation IDs

classes = np.array(["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega"])
scales = []
segment_class = []
for ob in seg_ObIDs:
    if ob in ob_state:
        segment_class.append(ob_state[ob])
    else:
        segment_class.append("Unknown")

In [ ]:
new_classification = shape_moments_GMM122_labels

Belloni_classes = np.array(["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega", "Unknown"])

comparison_matrix = np.zeros((len(np.unique(new_classification)), len(Belloni_classes)), dtype=int)

comparison_matrix_df = pd.DataFrame(comparison_matrix, columns=Belloni_classes, index=np.unique(new_classification))

for n_Bc, Belloni_class in enumerate(Belloni_classes):
    Belloni_class_indices = np.where(np.array(segment_class) == Belloni_class)[0]
    count_clusters_for_class = np.unique(np.take(new_classification, Belloni_class_indices), return_counts=True)
    for cluster_ind, cluster in enumerate(count_clusters_for_class[0]):
        comparison_matrix_df[Belloni_class][cluster] = count_clusters_for_class[1][cluster_ind]

In [ ]:
class_normalized_comparison_matrix_df=(comparison_matrix_df-comparison_matrix_df.min())/(comparison_matrix_df.max()-comparison_matrix_df.min())
known_comparison_matrix_df = comparison_matrix_df.drop(columns=['Unknown']).T
component_normalized_comparison_matrix_df = (known_comparison_matrix_df-known_comparison_matrix_df.min())/(known_comparison_matrix_df.max()-known_comparison_matrix_df.min())

In [ ]:
# find particularly class-homogeneous components

good_comps = {}
for comp in range(122):
    comp_counts = comparison_matrix_df.iloc[comp,:].sort_values(ascending=False)
    comp_counts_nonzero = comp_counts.where(comp_counts>0).dropna().astype(int)
    if comp_counts_nonzero.index[0] == "Unknown":
        comp_class_proportion = comp_counts_nonzero[1:]/comp_counts_nonzero[1:].sum()
        if comp_class_proportion[0]>0.999:
            good_comps[comp] = comp_counts_nonzero
    else:
        good_comps[comp] = comp_counts_nonzero
print(len(good_comps))
# dominated_comps = []
# for i,v in good_comps.items():
#     if v.index[1] == "chi":
#         dominated_comps.append(i)
# print(dominated_comps)

In [ ]:
# chi is >95% bar unknowns : (10), 3, 5, 7, 15, 24, 28, 37, 44, 53, 57, 60, 69, 76, 91, 103, 112
# kappa : (79),  4, 42, 78, 104 (87 has 90.8% kappa)
# theta : 25, 26, 71
# lambda : 66
# rho : 12, 18, 19, 20, 22, 23, 30, 38, 45, 49, 50, 54, 56, 63, 65, 74, 77, 86, 97, 107

print(good_comps)

In [ ]:
comparison_matrix_df.iloc[98]#/(comparison_matrix_df.iloc[87].sum()-2628)

In [ ]:
comp_list = []
for i,v in good_comps.items():
    if len(v)>1:
        comp_list.append(i)

In [ ]:
print(comp_list)

In [ ]:
comp_counts = comparison_matrix_df.iloc[7,:].sort_values(ascending=False)
comp_counts_nonzero = comp_counts.where(comp_counts>0).dropna().astype(int)
comp_counts_nonzero[1:]/comp_counts_nonzero[1:].sum()

In [ ]:
comparison_matrix_df.sum(axis=0)

In [ ]:
comparison_matrix_df.T.iloc[:,66]

In [ ]:
(0.002524/1.002524)*7617

In [ ]:
np.unique(shape_moments_GM114_labels, return_counts=True)[1][10]

In [ ]:
np.where(comparison_matrix_df.T.idxmax().values != "Unknown")

In [ ]:
plt.rcParams['figure.figsize'] = (6.97, 6.97*(1/3))
import seaborn as sns

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)



ax = sns.heatmap(class_normalized_comparison_matrix_df.T, xticklabels=True, yticklabels=True, cmap='coolwarm')#, linewidth=0.5)
# # ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 6)
# ax.tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="out")

ax.xaxis.set_major_locator(MultipleLocator(4))
ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))

# For the minor ticks, use no labels; default NullFormatter.
# ax.xaxis.set_minor_locator(MultipleLocator(1.5))
ax.xaxis.set_minor_locator(AutoMinorLocator())


# for label in ax.xaxis.get_majorticklabels():
#     label.set_transform(label.get_transform() + 0.5)
    
# for label in ax.xaxis.get_minorticklabels():
#     label.set_transform(label.get_transform() + 0.5)

# plt.title("Gaussian mixture components' populations in terms of classified data (component-wise min-maxed)")
plt.savefig("figures/GMM122vsBelloni_heatmap.png", dpi=300)
plt.show()

# Visualise Gaussian components' populations

In [ ]:
GMmodel_dir = "{}/GMM_122comp_20d_alldata_model_2020-08-30_11-42-38.pkl".format(data_dir)
with open(GMmodel_dir, 'rb') as f:
    GMmodel122 = pickle.load(f)

In [ ]:
with open("{}/shape_moments_GMM122_labels.pkl".format(data_dir), 'rb') as f: # output of LSTM autoencoder's decoder
    shape_moments_GMM122_labels = pickle.load(f)
    
# load light curve segments
with open('{}/468202_len128_s2_4cad_counts_errorfix.pkl'.format(data_dir), 'rb') as f:
    segments_counts = pickle.load(f)
    
# segments_dir = '{}/468202_len128_s2_4cad_counts_errorfix.pkl'.format(data_dir)
# errors_dir = '{}/468202_len128_s2_4cad_errors_errorfix.pkl'.format(data_dir)
recos_dir = "{}/reconstructions_from_model_2020-08-30_11-42-38.pkl".format(data_dir)

# with open(segments_dir, 'rb') as f:
#     segments = pickle.load(f)
# with open(errors_dir, 'rb') as f:
#     errors = pickle.load(f)
with open(recos_dir, 'rb') as f:
    segment_reconstructions = pickle.load(f)

segment_reconstructions= segment_reconstructions*np.std(segments_counts, axis=1) + np.mean(segments_counts, axis=1)

In [ ]:
np.max(shape_moments[:,-4:], axis=0)

In [ ]:
plt.close()

# counts_todo = [i for i,v in class_normalized_comparison_matrix_df["omega"].sort_values(ascending=False)[:10].items() if v>=0.5]

# if len(counts_todo) > 3:
#     how_many_to_plot = 10
# else:
how_many_to_plot = 15

plt.rcParams['figure.figsize'] = (3.32*3,3.32*(2)*3)
plt.rcParams.update({'font.size': 6})

component_counts = np.unique(shape_moments_GMM122_labels, return_counts=1)

max_stats = np.max(shape_moments[:,-4:],axis=0)
min_stats = np.min(shape_moments[:,-4:],axis=0)

for plot_component in [43,88,89,98,114]:#range(9):
    fig, axes = plt.subplots(nrows=how_many_to_plot, ncols=3)
    component_segment_indices = np.where(shape_moments_GMM122_labels == plot_component)[0]
    for plot_ind in range(how_many_to_plot):
        data = segments_counts[component_segment_indices[plot_ind]]
        reconstruction = segment_reconstructions[component_segment_indices[plot_ind]]
        axes[plot_ind,0].plot(np.array(list(range(128)))*4,data/1000)#, c="green", linewidth=0.5, zorder=-5)
        axes[plot_ind,0].plot(np.array(list(range(128)))*4, reconstruction/1000)
        axes[plot_ind,1].plot(np.array(list(range(128)))*4,data/1000)#, c="green", linewidth=0.5, zorder=-5)
        axes[plot_ind,1].plot(np.array(list(range(128)))*4, reconstruction/1000)
    #     axes[subplot, 1].plot(np.array(list(range(128)))*4,segments_counts[class_segments[subplot]])
    #     axes[subplot, 1].plot(np.array(list(range(128)))*4, reconstruction*np.std(data)+np.mean(data))
        axes[plot_ind, 1].set_ylim([0, 12])
        axes[plot_ind, 1].yaxis.tick_right()
        
#         bar_heights=np.zeros(4)
#         bar_heights[0] = np.mean(data)
#         bar_heights[1] = np.std(data)
#         bar_heights[2] = stats.skew(data)
#         bar_heights[3] = stats.kurtosis(data)

#         color = ["red" if x else "blue" for x in shape_moments[component_segment_indices[plot_ind],-4:]<0]
        bar_heights = (shape_moments[component_segment_indices[plot_ind],-4:]-min_stats)/ (max_stats-min_stats)
        axes[plot_ind, 2].bar(x=range(4), height=bar_heights)#, color=color)
        axes[plot_ind, 2].set_ylim([0,1])#[np.min(shape_moments[:,-4:]), np.max(shape_moments[:,-4:])])

    #     if class_name == "Unknown": 
    #         class_name = "??"
    #     else:
    #         class_name = r"$\{}$".format(class_name)
    #     axes[subplot, 1].text(x=510, y=9500, s=class_name)

        if plot_ind == 2:
            axes[plot_ind,0].set_ylabel("Rate (kcounts/s)", size=6)
        if plot_ind == 4:#plot_ind == 12 or plot_ind == 13
            axes[plot_ind,0].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
            axes[plot_ind,1].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
            axes[plot_ind,0].set_xlabel("Time (s)", size=6, x=1)

        else:
            axes[plot_ind,0].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=0, direction="in")
            axes[plot_ind,1].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=0, direction="in")

    plt.suptitle("component {}, population {}".format(plot_component, len(component_segment_indices)), y=1.01)
    plt.tight_layout()
    plt.show()


In [ ]:
GMmodel122.covariances_[[0,1,2]].shape

In [ ]:
filter_indices = [0,32,40,50,60,73,100,103,110,113,118]
for ind, component in enumerate(GMmodel122.covariances_[filter_indices]):
#     print(GMmodel122.means_[filter_indices][ind])
    plt.bar(x=np.concatenate((np.array(range(16)), np.array(range(4))+17)), height= np.diagonal(component))
    plt.title("{}".format(filter_indices[ind]))
    plt.ylim((0,3))
    plt.show()

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(GMmodel122.means_)


embedding = reducer.transform(GMmodel122.means_)


In [ ]:
plt.rcParams['figure.figsize'] = (3.32,3.32)

zorders = [-1]*122
labels = ["Other"]*122

s=5

plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)


filter_indices = [6,10,16,21,22,24,37,38,41,45,56,57,59,63,79,81,82,83,90,94,96,105,108,115,119,121]# regular peaks

plt.scatter(embedding[filter_indices,0], embedding[filter_indices,1], c="orange", label= "periodic flare", s=s)


colour_filter = ["orange" if x in filter_indices else "black" for x in range(122)]
zorders = [1 if ind in filter_indices else x for ind, x in enumerate(zorders)]
labels = ["Periodic flare" if ind in filter_indices else x for ind, x in enumerate(zorders)]

filter_indices = [0,32,40,50,60,73,100,103,110,113,118]# mid-flats

plt.scatter(embedding[filter_indices,0], embedding[filter_indices,1], c="magenta", label= "mid random", s=s)


colour_filter = ["magenta" if ind in filter_indices else x for ind, x in enumerate(colour_filter)]
zorders = [1 if ind in filter_indices else x for ind, x in enumerate(zorders)]
labels = ["Mid random" if ind in filter_indices else x for ind, x in enumerate(zorders)]

filter_indices = [1,2,4,5,7,12,14,18,27, 30,31,47,49,51,54,67,71,76,80,99,106,109]# low-flats

plt.scatter(embedding[filter_indices,0], embedding[filter_indices,1], c="red", label= "low random", s=s)


filter_indices = [8,11, 52,66,68,117]# low-flats

plt.scatter(embedding[filter_indices,0], embedding[filter_indices,1], c="cyan", label= "+ve gradient", s=s)

filter_indices = [43,88,89,98,114]
plt.scatter(embedding[filter_indices,0], embedding[filter_indices,1], c="khaki", label= "\'square wave\'", s=s)

colour_filter = ["cyan" if ind in filter_indices else x for ind, x in enumerate(colour_filter)]
zorders = [1 if ind in filter_indices else x for ind, x in enumerate(zorders)]
labels = ["Low random" if ind in filter_indices else x for ind, x in enumerate(zorders)]

plt.xlabel("UMAP axis 1")
plt.ylabel("UMAP axis 2")

# filter_indices = [19,34,39,70]# irregular peaks

# colour_filter = ["red" if ind in filter_indices else x for ind, x in enumerate(colour_filter)]

# plt.scatter(embedding[:,0], embedding[:,1], c=colour_filter)
plt.legend()
# plt.savefig('figures/UMAP_component_means.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)


plt.show()

In [ ]:
len((0,32,40,50,60,73,100,103,110,113,118))

In [ ]:
means = []
for plot_component in (1,2,4,5,7,12,14,18,27, 30,31,47,49,51,54,67,71,76,80,99,106,109,111,112,116):#range(9):
    component_segment_indices = np.where(shape_moments_GMM122_labels == plot_component)[0]
    mean = np.mean(segments_counts[component_segment_indices], axis=1)
    means = np.concatenate((means, mean.flatten()))
    
means2 = []
for plot_component in (0,32,40,50,60,73,100,103,110,113,118):#range(9):
    component_segment_indices = np.where(shape_moments_GMM122_labels == plot_component)[0]
    mean = np.mean(segments_counts[component_segment_indices], axis=1)
    means2 = np.concatenate((means2, mean.flatten()))

In [ ]:
plt.hist(means)
plt.hist(means2, alpha=0.5)

plt.show()

In [ ]:
np.mean(means2)

# Build a dataframe containing counts of light curve segments in each Gaussian mixture component for each observation
intended shape of the data frame: [1738, 122]

In [ ]:

with open("{}/ObID_GaussComps_dict_122_chrono.pkl".format(data_dir), 'rb') as f: # same as ObID_GaussComps_dict_122 below, but segments are in chronological order
    ObID_GaussComps_dict_122_chrono = pickle.load(f)

In [ ]:
with open("{}/shape_moments_GMM122_labels.pkl".format(data_dir), 'rb') as f: # output of LSTM autoencoder's decoder
    shape_moments_GMM122_labels = pickle.load(f)

In [ ]:
# load (shuffled) list of unique IDs for the 468202 light curve segments. Format: observationID_segmentIndex,
# i.e. ['96701-01-48-00_3','20402-01-02-02_122','70703-01-01-000_1420', ...]
with open('{}/468202_len128_s2_4cad_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    seg_ids = pickle.load(f)

# make a list of observation ids; for each of the 468202 segments, get the ID of the observation that it comes from
#i.e. ['96701-01-48-00','20402-01-02-02', '70703-01-01-000',
seg_ObIDs = [seg.split("_")[0] for seg in seg_ids] # gets rid of the within-observation segment indices and creates a degenerate list of observation IDs

In [ ]:
# make a dict that groups indices of segments of the same observation 
# i.e. where each observation id can be found in seg_ObIDs
#i.e. ObID_SegIndices_dict == {'10258-01-01-00': [916, 949, 1046...467528, 467578], ....}
ObID_SegIndices_dict = {key:[] for key in np.unique(seg_ObIDs)}
for ID_index, ObID in enumerate(seg_ObIDs):
    ObID_SegIndices_dict.setdefault(ObID, []).append(ID_index)

In [ ]:
# make a dictionary of Gaussian component labels instead of segment indices  
#i.e. ObID_GaussComps_dict_500 == {'10258-01-01-00': [401, 433, 382...101, 152], ....}
ObID_GaussComps_dict_122 = {}
for ObID, Indices in ObID_SegIndices_dict.items():
    ObID_GaussComps_dict_122[ObID] = [shape_moments_GMM122_labels[ind] for ind in Indices]

In [ ]:
# make a data frame containing the counts of light curve segments in each of the Gaussian components, for each observation
obs_component_counts_df_122 = pd.DataFrame(np.zeros((len(ObID_GaussComps_dict_122),len(np.unique(shape_moments_GMM122_labels)))), index=np.unique(seg_ObIDs), columns=list(range(122)), dtype=int)

In [ ]:
# populate the data frame
for ObID, GaussComps in ObID_GaussComps_dict_122.items():
    for comp_id, comp_count in np.array(np.unique(GaussComps, return_counts=True)).T:
        obs_component_counts_df_122.loc[ObID][comp_id] = comp_count
obs_component_counts_df_122

In [ ]:
#sanity check; count of the number of segments in each Gaussian component, as returned by the Gaussian_mixture_model.predict() method,
#matches the  count in the columns of the data frame, so we transferred the counts from list to dataframe successfully
print((np.unique(shape_moments_GMM122_labels, return_counts= True)[1] == obs_component_counts_df_122.sum().values).all())
#total number of segment counts matches the size of our dataset
print(obs_component_counts_df_122.sum().sum())
#each component has at least 59 counts in it, so I didn't mis-name "shape_moments_GM114_labels" as "shape_moments_GM500_labels" 
print(obs_component_counts_df_122.sum().values.min())

In [ ]:
# apply min-max normalisation to values in each row of the data frame
from sklearn.preprocessing import Normalizer
normalized_obs_component_counts_df_122 = pd.DataFrame(Normalizer(norm='max').fit_transform(obs_component_counts_df_122), index=np.unique(seg_ObIDs), columns=list(range(122)))
normalized_obs_component_counts_df_122

# Apply center log ratio transformation to the counts

In [ ]:
from scipy.stats.mstats import gmean
# multiply each value by 10 to set true counts apart from pseudo-counts, then replace all zeros with 0.5 (half of the lowest non-zero value)
pseudo_counts = (obs_component_counts_df_122).replace(to_replace=0, value=0.5) 
# apply closure normalisation; each row should sum up to 1
closed_pseudo_counts = pseudo_counts.div(np.sum(pseudo_counts, axis=1), axis="rows")
#divide each row by its geometric mean and take a natural log. this is the CLR transformation, inverse of a softmax
scaled_pseudo_counts = closed_pseudo_counts.div(gmean(closed_pseudo_counts, axis=1), axis="rows")
pseudo_CLR_transformed = np.log(scaled_pseudo_counts) # pseudo-counts are now negative, and real counts positive

In [ ]:
pseudo_CLR_transformed

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=20)
# clusterer.fit(pseudo_CLR_transformed)

# clusterer.labels_

In [ ]:
reducer = umap.UMAP(random_state=2, n_components=12, n_neighbors=10, min_dist=0.)#300)
reducer.fit(pseudo_CLR_transformed)
embedding = reducer.transform(pseudo_CLR_transformed)

In [ ]:
clusterer.fit(embedding)

np.unique(clusterer.labels_, return_counts=True)

In [ ]:
reducer2 = umap.UMAP(random_state=2, n_components=2, n_neighbors=10, min_dist=0.)#300)
reducer2.fit(embedding)
embedding = reducer.transform(embedding)

In [ ]:
cluster_inds = np.where(clusterer.labels_==-1)[0]
cluster_data = np.take(embedding,cluster_inds, axis=0)
plt.scatter(cluster_data[:,0], cluster_data[:,1], c="black", s=0.5)
for cluster in np.unique(clusterer.labels_)[1:]:
    cluster_inds = np.where(clusterer.labels_==cluster)[0]
    cluster_data = np.take(embedding,cluster_inds, axis=0)
    plt.scatter(cluster_data[:,0], cluster_data[:,1], s=0.5)
plt.show()

In [ ]:
from scipy.stats.mstats import gmean
# multiply each value by 10 to set true counts apart from pseudo-counts, then replace all zeros with 0.5 (half of the lowest non-zero value)
pseudo_counts = (obs_component_counts_df_122).replace(to_replace=0, value=0.5) 
# apply closure normalisation; each row should sum up to 1
closed_pseudo_counts = pseudo_counts.div(np.sum(pseudo_counts, axis=1), axis="rows")
#divide each row by its geometric mean and take a natural log. this is the CLR transformation, inverse of a softmax
scaled_pseudo_counts = closed_pseudo_counts.div(gmean(closed_pseudo_counts, axis=1), axis="rows")
pseudo_CLR_transformed = np.log(scaled_pseudo_counts) # pseudo-counts are now negative, and real counts positive

In [ ]:
obs_component_counts_df_122

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 5.0)
plt.plot((obs_component_counts_df_122 == 0).astype(int).sum(axis=0).values)
plt.ylim((0,1738))
plt.ylabel("Number of observations with a zero")
plt.xlabel("Feature ID")
plt.show()

In [ ]:
# multiplicative strategy from Martin-Fernandez
closed_df = obs_component_counts_df_122.div(np.sum(obs_component_counts_df_122, axis=1), axis="rows")
deltaj = 0.65*closed_df[closed_df>0].min(skipna=True).min()
sum_no_zeros_in_rows = (obs_component_counts_df_122 == 0).astype(int).sum(axis=1)
scaled_nonzero = closed_df.multiply((1-sum_no_zeros_in_rows*deltaj), axis="rows")
zeros_replaced =scaled_nonzero.replace(to_replace=0, value=deltaj)
scaled_xs = zeros_replaced.div(gmean(zeros_replaced, axis=1), axis="rows")
pseudo_CLR_transformed_x = np.log(scaled_xs)

In [ ]:
# not sure
deltaj = 0.65/obs_component_counts_df_122.iloc[0,:].sum()
x = obs_component_counts_df_122.div(np.sum(obs_component_counts_df_122, axis=1), axis="rows")
sum_no_zeros_in_rows = (x == 0).astype(int).sum(axis=1)
x = x.multiply((1-sum_no_zeros_in_rows*deltaj), axis="rows")
x=x.replace(to_replace=0, value=deltaj)
scaled_xs = closed_pseudo_counts.div(gmean(x, axis=1), axis="rows")
pseudo_CLR_transformed_x = np.log(scaled_xs)

In [ ]:
x.sum(axis=1)

In [ ]:
plt.close()

plt.rcParams['figure.figsize'] = (30,30)
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True)

tril = np.array(np.tril_indices(5)).T

data = pseudo_CLR_transformed_x

for x,y in tril:
    axes[x,y].scatter(data.iloc[:,x],data.iloc[:,y])
#     axes[x,y].set_ylim((-0.1,1))
#     axes[x,y].set_xlim((-0.1,1))
    axes[x,y].text(0.99, 0.99, np.round(np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1], decimals=3), ha='right', va='top', transform=axes[x,y].transAxes, size=30)

plt.show()


In [ ]:
plt.close()

plt.rcParams['figure.figsize'] = (30,30)
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True)

tril = np.array(np.tril_indices(5)).T

data = obs_component_counts_df_122

for x,y in tril:
    axes[x,y].scatter(data.iloc[:,x],data.iloc[:,y])
#     axes[x,y].set_ylim((-0.1,1))
#     axes[x,y].set_xlim((-0.1,1))
    axes[x,y].text(0.99, 0.99, np.round(np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1], decimals=3), ha='right', va='top', transform=axes[x,y].transAxes, size=30)

plt.show()


In [ ]:
reducer = umap.UMAP(random_state=2, n_components=20, n_neighbors=300)#300)
reducer.fit(pseudo_CLR_transformed_x)
embedding = reducer.transform(pseudo_CLR_transformed_x)

plt.rcParams['figure.figsize'] = [5,5]

plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

data = embedding
metric = "euclidean"

linkage = sch.linkage(data, method="ward", metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]

no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(calinski_harabasz_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(4.84678277, c="black")
# ax1.axvline(4.291764, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, curve='convex', direction='decreasing')
print(np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False).shape)

plt.show()


In [ ]:
# reducer = umap.UMAP(random_state=0, n_components=2, n_neighbors=15)#300)
# reducer.fit(pseudo_CLR_transformed)
# embedding = reducer.transform(pseudo_CLR_transformed)

# plt.rcParams['figure.figsize'] = [5,5]

# plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
# plt.show()

data = pseudo_CLR_transformed
metric = "euclidean"

linkage = sch.linkage(data, method="ward", metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]

no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(calinski_harabasz_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(4.84678277, c="black")
# ax1.axvline(4.291764, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, curve='convex', direction='decreasing')
print(np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False).shape)

plt.show()


In [ ]:
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False)

In [ ]:
kn.knee

In [ ]:
# from sklearn.metrics import silhouette_score
# from sklearn.metrics import calinski_harabasz_score
# plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
# y_top_limit = linkage[-1][2]
# y_bottom_limit = linkage[0][2]
# # no_clusters = []
# # silhouette_scores0 = []
# # for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
# #     clusters = sch.fcluster(linkage, distance, criterion='distance')
# #     silhouette_scores0.append(silhouette_score(data, clusters))#, metric=metric, random_state=0))
# #     no_clusters.append(len(np.unique(clusters)))

# fig, ax1 = plt.subplots()
# plt.rcParams['figure.figsize'] = [3.32,3.32]
# ax1.axvline(17.834425221673342, c="cyan")
# ax1.axhline(113, c="cyan")

# ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# # ax2 = ax1.twinx()
# # ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
# ax1.set_ylabel("No. clusters")
# # ax2.set_ylabel("Silhouette score", c="cyan")
# ax1.set_xlabel("Distance threshold")
# print("Best 20 cluster numbers: ", np.array(no_clusters0)[np.argsort(silhouette_scores0)[-20:]])
# print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
# # plt.savefig('figures/agglomerative_knee_113.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

# plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
# no_clusters = []
# silhouette_scores0 = []
# for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
#     clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores0.append(silhouette_score(data, clusters))#, metric=metric, random_state=0))
#     no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [3.32,3.32]
# ax1.axvline(kn.knee, c="cyan")
# ax1.axhline(53, c="cyan")

ax1.axvline(kn.knee, c="cyan")
ax1.axhline(113, c="cyan")

ax1.text(0.90,0.6,"Knee distance: {:.3f}\n Knee no. clusters: {}".format(kn.knee, np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False).shape[0]),
                    ha='right', va='top', transform=ax1.transAxes, size=6)

ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# ax2 = ax1.twinx()
# ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters")
# ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Euclidean distance threshold")
# print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores0)[-20:]])
# print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.savefig('figures/agglomerative_knee_113.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()


In [ ]:
reducer = umap.UMAP(random_state=0, min_dist=0., n_neighbors=300)
reducer.fit(pseudo_CLR_transformed_x)
embedding = reducer.transform(pseudo_CLR_transformed_x)

In [ ]:
print("UMAP of CLR transformed data, Euclidean distance")
plt.rcParams['figure.figsize'] = [5,5]

plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
UMAP_n_components = [2,5,10,20]
UMAP_n_neighbors = [15, 300, 1737]
UMAP_seeds = list(range(100))

UMAP_mappers_list = []
linkage_list = []
knee_distance_list = []
knee_no_cluster_list = []
results_list = []

results_list.append(("List of hyperparameters: ", "UMAP_n_component", UMAP_n_components, "UMAP_n_neightbor", UMAP_n_neighbors, "UMAP_seed", UMAP_seeds, 
                     "Saving number of clusters at knee point of Euclidean distance threshold curve for ward agglomerative clustering of UMAPed data"))


for UMAP_n_component in UMAP_n_components:
    for UMAP_n_neightbor in UMAP_n_neighbors:
        for UMAP_seed in UMAP_seeds:
#             pca = PCA(n_components=PCA_var)
#             pca.fit(pseudo_CLR_transformed)
#             pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
            UMAP_reducer = umap.UMAP(random_state=UMAP_seed, n_neighbors=UMAP_n_neightbor,n_components=UMAP_n_component)
            #2,min_dist=0.1, n_neighbors=1737, n_components=30, n_epochs=1000, transform_queue_size=10)
            UMAP_reducer.fit(pseudo_CLR_transformed)
            UMAP_embedding = UMAP_reducer.transform(pseudo_CLR_transformed)

            data = UMAP_embedding
            metric = "euclidean"
            method="ward"
            linkage = sch.linkage(data, method=method, metric=metric)
            y_top_limit = linkage[-1][2]
            y_bottom_limit = linkage[0][2]
            no_clusters = []
            for distance in np.linspace(y_bottom_limit,y_top_limit,1000):
                clusters = sch.fcluster(linkage, distance, criterion='distance')
                no_clusters.append(len(np.unique(clusters)))

            from kneed import KneeLocator
            kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000), no_clusters, curve='convex', direction='decreasing')
            knee_no_clusters = len(np.unique(sch.fcluster(linkage, kn.knee, criterion='distance')))

            results_list.append(((UMAP_n_component, UMAP_n_neightbor, UMAP_seed, knee_no_clusters)))
            print(results_list[-1])
                
with open("UMAP_agglo_clustering_results4x3x100.pkl", 'wb') as f:
    pickle.dump(results_list, f)

In [ ]:
results_list

In [ ]:
# with open("UMAP_agglo_clustering_results.pkl", 'wb') as f:
#     pickle.dump(results_list, f)

In [ ]:
results_arr[:,:,:,:].flatten().shape

In [ ]:
#"UMAP_n_component", "UMAP_n_neightbor", "UMAP_seed"), ("UMAP_reducer", "linkage", "kn.knee", "knee_no_clusters"
# UMAP_n_components = [2,5,10,20]
# UMAP_n_neighbors = [15, 300, 1737]
# UMAP_seeds = list(range(10))

results_arr = np.zeros((4,3,100)).flatten()
for res_ind, res in enumerate(results_list[1:]):
    results_arr[res_ind] = res[-1]
#     print(res[0], res[1][-1])
results_arr = results_arr.reshape((4,3,100))

In [ ]:
np.max(results_arr)

In [ ]:
# test_this=results_arr[3,3,0,:]

# UMAP_n_components = [2,5,10,20]
# UMAP_n_neighbors = [15, 300, 1737]

fig, axes = plt.subplots(nrows=2, ncols=2)
axes = axes.flatten()

for comp_ind in range(4):
    for neigh_ind in range(3):
        mean_no_clusters = np.mean(results_arr[comp_ind,neigh_ind,:])
        axes[comp_ind].hist(results_arr[comp_ind,neigh_ind,:].flatten(), alpha=0.5, range=(30,100),
                            bins=20, label="{} ({:.2f})".format(UMAP_n_neighbors[neigh_ind], mean_no_clusters))#, range=(0,1737), bins=100 
        axes[comp_ind].set_xlim((30,100))
        axes[comp_ind].set_ylim((0,50))
    axes[comp_ind].text(0.90,0.6,"No. UMAP\ncomponents: {}".format(UMAP_n_components[comp_ind]),
                        ha='right', va='top', transform=axes[comp_ind].transAxes, size=6)
    axes[comp_ind].legend(loc='upper right', title="No. neighbours (mean)")
    print( np.mean(results_arr[comp_ind,:,:]))
axes[2].set_ylabel("Population", size=6, y=1)
axes[2].set_xlabel("No. clusters at the knee point", size=6, x=1)
axes.reshape((2,2))
plt.savefig('figures/No_cluster_histograms.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)
plt.show()
    
# plt.hist(results_arr[:,:,:,:].flatten(), bins=100, range=(0,1737))
# plt.hist(test_this.flatten(), alpha=0.8, bins=100,range=(0,1737))



# plt.show()
# plt.hist(results_arr[:,:,:,:].flatten(), bins=100,range=(0,1737))
# plt.hist(test_this.flatten(), alpha=0.8, bins=100,range=(0,1737))
# plt.show()

In [ ]:
import scipy.cluster.hierarchy as sch


reducer = umap.UMAP(random_state=0, n_components=20, n_neighbors=300)
reducer.fit(pseudo_CLR_transformed)
embedding = reducer.transform(pseudo_CLR_transformed)
s=0.5

plt.rcParams['figure.figsize'] = [5,5]

plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

data = embedding
metric = "euclidean"

linkage = sch.linkage(data, method="ward", metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]

no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(calinski_harabasz_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(4.84678277, c="black")
# ax1.axvline(4.291764, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, curve='convex', direction='decreasing')
print(np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False).shape)

plt.show()


In [ ]:
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False)

In [ ]:
kn.knee

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
# no_clusters = []
# silhouette_scores0 = []
# for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
#     clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores0.append(silhouette_score(data, clusters))#, metric=metric, random_state=0))
#     no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [3.32,3.32]
ax1.axvline(kn.knee, c="cyan")
ax1.axhline(53, c="cyan")

ax1.text(0.90,0.6,"Knee distance: {:.3f}\n Knee no. clusters: {}".format(kn.knee, np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False).shape[0]),
                    ha='right', va='top', transform=ax1.transAxes, size=6)

ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# ax2 = ax1.twinx()
# ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters")
# ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Euclidean distance threshold")
# print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores0)[-20:]])
# print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.savefig('figures/agglomerative_knee.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()


In [ ]:
kn.knee

In [ ]:
data = embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0, color_threshold=kn.knee)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Euclidean distance threshold", size=6, x=1)
axis.set_xlabel("Leaves of the dendogram", size=6)
plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'))

In [ ]:
reducer = umap.UMAP(random_state=0, n_components=2, n_neighbors=300)
reducer.fit(pseudo_CLR_transformed)
embedding = reducer.transform(pseudo_CLR_transformed)

In [ ]:
# reducer = umap.UMAP(random_state=0, n_components=2, n_neighbors=300)
# reducer.fit(embedding)
# embedding = reducer.transform(embedding)

In [ ]:
clusters53 = sch.fcluster(linkage, kn.knee, criterion='distance')-1
clusters53_split = np.split(np.argsort(clusters53), indices_or_sections=np.cumsum(np.unique(clusters53, return_counts=True)[1][:-1]))

plt.rcParams['figure.figsize'] = [3.32,3.32]

print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
# plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
for cluster in clusters53_split:
    plt.scatter(embedding[:,0][cluster], embedding[:,1][cluster], s=1)
plt.xlabel("UMAP axis 1")
plt.ylabel("UMAP axis 2")

# plt.savefig('figures/UMAP_agglomerative.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
plt.hist([x for x in pseudo_CLR_transformed.to_numpy().flatten() if x>0])
plt.show()

In [ ]:
plt.hist([x for x in obs_component_counts_df_122.to_numpy().flatten() if x>0])
plt.show()

In [ ]:
reducer = umap.UMAP(random_state=0)
reducer.fit(pseudo_CLR_transformed)
embedding = reducer.transform(pseudo_CLR_transformed)

In [ ]:
print("UMAP of CLR transformed data, Euclidean distance")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
for rng in range(10):
    print("RNG: ", rng)

    from sklearn.decomposition import PCA
    pca = PCA(n_components=122)
    pca.fit(pseudo_CLR_transformed)

    print(pca.explained_variance_ratio_.shape)

    reducer = umap.UMAP(random_state=rng, n_components=20)
    reducer.fit(pca.transform(pseudo_CLR_transformed))
    embedding = reducer.transform(pca.transform(pseudo_CLR_transformed))

    plt.rcParams['figure.figsize'] = [5,5]

    print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
    plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
    plt.show()

    data = embedding
    metric = "euclidean"

    linkage = sch.linkage(data, method="ward", metric=metric)
    plt.rcParams['figure.figsize'] = (6.97, 3)
    plt.rcParams.update({'font.size': 6})

    dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
    axis = plt.gca()
    # axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
    axis.set_xticks([])
    axis.set_xticklabels([])
    axis.set_ylabel("euclidean distance", size=6, x=1)

    # plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


    plt.show()

    from sklearn.metrics import silhouette_score
    from sklearn.metrics import calinski_harabasz_score

    plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


    y_top_limit = linkage[-1][2]
    y_bottom_limit = linkage[0][2]

    no_clusters = []
    silhouette_scores = []
    for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
        clusters = sch.fcluster(linkage, distance, criterion='distance')
        silhouette_scores.append(calinski_harabasz_score(data, clusters))#, metric=metric, random_state=0))
        no_clusters.append(len(np.unique(clusters)))

    fig, ax1 = plt.subplots()


    plt.rcParams['figure.figsize'] = [5,5]
    # plt.axhline(114, c="cyan")
    # ax1.axvline(4.84678277, c="black")
    # ax1.axvline(4.291764, c="black")
    # ax1.axhline(60, c="black")
    ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

    ax2 = ax1.twinx()
    ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
    # plt.title("No. of clusters as the function of distance for hierarchical clustering")
    # ax3 = ax1.twinx()
    # ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
    # ax4 = ax1.twinx()
    # ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

    ax1.set_ylabel("No. clusters", c="magenta")
    ax2.set_ylabel("Silhouette score", c="cyan")
    ax1.set_xlabel("Distance threshold")
    # plt.ylim((0,50))
    # plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)
    print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
    print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
    from kneed import KneeLocator
    kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, curve='convex', direction='decreasing')
    print(np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False).shape)

    plt.show()


In [ ]:
split_this = sch.fcluster(linkage, 3.27677289, criterion='distance')
clusters53_split = np.split(np.argsort(split_this), indices_or_sections=np.cumsum(np.unique(split_this, return_counts=True)[1][:-1]))

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]

print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
# plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
for cluster in clusters53_split:
    plt.scatter(embedding[:,0][cluster], embedding[:,1][cluster], s=1)
plt.show()

In [ ]:
data = pca.transform(pseudo_CLR_transformed)
metric = "euclidean"

linkage = sch.linkage(data, method="ward", metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]

no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))


In [ ]:
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(4.84678277, c="black")
# ax1.axvline(4.291764, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
split_this = sch.fcluster(linkage, 61.96563317, criterion='distance')
clusters29_split = np.split(np.argsort(split_this), indices_or_sections=np.cumsum(np.unique(split_this, return_counts=True)[1][:-1]))

In [ ]:
n=0

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]

print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
for cluster in clusters29_split[n]:
    plt.scatter(embedding[:,0][cluster], embedding[:,1][cluster], s=1, c="cyan")
plt.show()
n+=1

In [ ]:
pseudo_CLR_transformed

In [ ]:
reducer = umap.UMAP(random_state=1, n_neighbors=1737,min_dist=0.0, n_components=50, n_epochs=10000, transform_queue_size=10)
reducer.fit(pseudo_CLR_transformed)
embedding = reducer.transform(pseudo_CLR_transformed)

In [ ]:
data = embedding
metric = "euclidean"

linkage = sch.linkage(data, method="ward", metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]

no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))


In [ ]:
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(4.84678277, c="black")
# ax1.axvline(4.291764, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bboobcy z marsa też są spoko, ale na wenus znaleziono sygnatury fosforowodorów; powstały albo przez jakieś ekstremalne procesy chemiczne albo w brzuszkach kosmitów PogChamp_inches = 'tight',pad_inches = 0)
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
reducer = umap.UMAP(random_state=0, n_neighbors=1737,min_dist=0.0, n_components=5, n_epochs=10000, transform_queue_size=10)
reducer.fit(pseudo_CLR_transformed)
embedding = reducer.transform(pseudo_CLR_transformed)

In [ ]:
data = embedding
metric = "euclidean"

linkage = sch.linkage(data, method="ward", metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]

no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))


In [ ]:
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(4.84678277, c="black")
# ax1.axvline(4.291764, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bboobcy z marsa też są spoko, ale na wenus znaleziono sygnatury fosforowodorów; powstały albo przez jakieś ekstremalne procesy chemiczne albo w brzuszkach kosmitów PogChamp_inches = 'tight',pad_inches = 0)
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
Init signature:
umap.UMAP(
    n_neighbors=15,
    n_components=2,
    metric='euclidean',
    metric_kwds=None,
    output_metric='euclidean',
    output_metric_kwds=None,
    n_epochs=None,
    learning_rate=1.0,
    init='spectral',
    min_dist=0.1,
    spread=1.0,
    low_memory=False,
    set_op_mix_ratio=1.0,
    local_connectivity=1.0,
    repulsion_strength=1.0,
    negative_sample_rate=5,
    transform_queue_size=4.0,
    a=None,
    b=None,
    random_state=None,
    angular_rp_forest=False,
    target_n_neighbors=-1,
    target_metric='categorical',
    target_metric_kwds=None,
    target_weight=0.5,
    transform_seed=42,
    force_approximation_algorithm=False,
    verbose=False,
    unique=False,
)
Docstring:     
Uniform Manifold Approximation and Projection

Finds a low dimensional embedding of the data that approximates
an underlying manifold.

Parameters
----------
n_neighbors: float (optional, default 15)
    The size of local neighborhood (in terms of number of neighboring
    sample points) used for manifold approximation. Larger values
    result in more global views of the manifold, while smaller
    values result in more local data being preserved. In general
    values should be in the range 2 to 100.

n_components: int (optional, default 2)
    The dimension of the space to embed into. This defaults to 2 to
    provide easy visualization, but can reasonably be set to any
    integer value in the range 2 to 100.

metric: string or function (optional, default 'euclidean')
    The metric to use to compute distances in high dimensional space.
    If a string is passed it must match a valid predefined metric. If
    a general metric is required a function that takes two 1d arrays and
    returns a float can be provided. For performance purposes it is
    required that this be a numba jit'd function. Valid string metrics
    include:
        * euclidean
        * manhattan
        * chebyshev
        * minkowski
        * canberra
        * braycurtis
        * mahalanobis
        * wminkowski
        * seuclidean
        * cosine
        * correlation
        * haversine
        * hamming
        * jaccard
        * dice
        * russelrao
        * kulsinski
        * ll_dirichlet
        * hellinger
        * rogerstanimoto
        * sokalmichener
        * sokalsneath
        * yule
    Metrics that take arguments (such as minkowski, mahalanobis etc.)
    can have arguments passed via the metric_kwds dictionary. At this
    time care must be taken and dictionary elements must be ordered
    appropriately; this will hopefully be fixed in the future.

n_epochs: int (optional, default None)
    The number of training epochs to be used in optimizing the
    low dimensional embedding. Larger values result in more accurate
    embeddings. If None is specified a value will be selected based on
    the size of the input dataset (200 for large datasets, 500 for small).

learning_rate: float (optional, default 1.0)
    The initial learning rate for the embedding optimization.

init: string (optional, default 'spectral')
    How to initialize the low dimensional embedding. Options are:
        * 'spectral': use a spectral embedding of the fuzzy 1-skeleton
        * 'random': assign initial embedding positions at random.
        * A numpy array of initial embedding positions.

min_dist: float (optional, default 0.1)
    The effective minimum distance between embedded points. Smaller values
    will result in a more clustered/clumped embedding where nearby points
    on the manifold are drawn closer together, while larger values will
    result on a more even dispersal of points. The value should be set
    relative to the ``spread`` value, which determines the scale at which
    embedded points will be spread out.

spread: float (optional, default 1.0)
    The effective scale of embedded points. In combination with ``min_dist``
    this determines how clustered/clumped the embedded points are.

low_memory: bool (optional, default False)
    For some datasets the nearest neighbor computation can consume a lot of
    memory. If you find that UMAP is failing due to memory constraints
    consider setting this option to True. This approach is more
    computationally expensive, but avoids excessive memory use.

set_op_mix_ratio: float (optional, default 1.0)
    Interpolate between (fuzzy) union and intersection as the set operation
    used to combine local fuzzy simplicial sets to obtain a global fuzzy
    simplicial sets. Both fuzzy set operations use the product t-norm.
    The value of this parameter should be between 0.0 and 1.0; a value of
    1.0 will use a pure fuzzy union, while 0.0 will use a pure fuzzy
    intersection.

local_connectivity: int (optional, default 1)
    The local connectivity required -- i.e. the number of nearest
    neighbors that should be assumed to be connected at a local level.
    The higher this value the more connected the manifold becomes
    locally. In practice this should be not more than the local intrinsic
    dimension of the manifold.

repulsion_strength: float (optional, default 1.0)
    Weighting applied to negative samples in low dimensional embedding
    optimization. Values higher than one will result in greater weight
    being given to negative samples.

negative_sample_rate: int (optional, default 5)
    The number of negative samples to select per positive sample
    in the optimization process. Increasing this value will result
    in greater repulsive force being applied, greater optimization
    cost, but slightly more accuracy.

transform_queue_size: float (optional, default 4.0)
    For transform operations (embedding new points using a trained model_
    this will control how aggressively to search for nearest neighbors.
    Larger values will result in slower performance but more accurate
    nearest neighbor evaluation.

a: float (optional, default None)
    More specific parameters controlling the embedding. If None these
    values are set automatically as determined by ``min_dist`` and
    ``spread``.
b: float (optional, default None)
    More specific parameters controlling the embedding. If None these
    values are set automatically as determined by ``min_dist`` and
    ``spread``.

random_state: int, RandomState instance or None, optional (default: None)
    If int, random_state is the seed used by the random number generator;
    If RandomState instance, random_state is the random number generator;
    If None, the random number generator is the RandomState instance used
    by `np.random`.

metric_kwds: dict (optional, default None)
    Arguments to pass on to the metric, such as the ``p`` value for
    Minkowski distance. If None then no arguments are passed on.

angular_rp_forest: bool (optional, default False)
    Whether to use an angular random projection forest to initialise
    the approximate nearest neighbor search. This can be faster, but is
    mostly on useful for metric that use an angular style distance such
    as cosine, correlation etc. In the case of those metrics angular forests
    will be chosen automatically.

target_n_neighbors: int (optional, default -1)
    The number of nearest neighbors to use to construct the target simplcial
    set. If set to -1 use the ``n_neighbors`` value.

target_metric: string or callable (optional, default 'categorical')
    The metric used to measure distance for a target array is using supervised
    dimension reduction. By default this is 'categorical' which will measure
    distance in terms of whether categories match or are different. Furthermore,
    if semi-supervised is required target values of -1 will be trated as
    unlabelled under the 'categorical' metric. If the target array takes
    continuous values (e.g. for a regression problem) then metric of 'l1'
    or 'l2' is probably more appropriate.

target_metric_kwds: dict (optional, default None)
    Keyword argument to pass to the target metric when performing
    supervised dimension reduction. If None then no arguments are passed on.

target_weight: float (optional, default 0.5)
    weighting factor between data topology and target topology. A value of
    0.0 weights entirely on data, a value of 1.0 weights entirely on target.
    The default of 0.5 balances the weighting equally between data and target.

transform_seed: int (optional, default 42)
    Random seed used for the stochastic aspects of the transform operation.
    This ensures consistency in transform operations.

verbose: bool (optional, default False)
    Controls verbosity of logging.

unique: bool (optional, default False)
    Controls if the rows of your data should be uniqued before being
    embedded.  If you have more duplicates than you have n_neighbour
    you can have the identical data points lying in different regions of
    your space.  It also violates the definition of a metric.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1, n_neighbors=1737,min_dist=0.0, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1, n_neighbors=1737,min_dist=0.0, n_components=2, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1, n_neighbors=100,min_dist=0.0, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1, n_neighbors=50,min_dist=0.0, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1)#, n_neighbors=50,min_dist=0.0, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1,min_dist=0.0)#, n_neighbors=50,min_dist=0.0, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1, n_neighbors=1737, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
7.254446096396663from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1, n_neighbors=1737)#, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=0)#, n_neighbors=1737)#, n_components=5, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=0,min_dist=0.1, n_neighbors=1737, n_components=10, n_epochs=10000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
# silhouette_scores0 = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores0.append(davies_bouldin_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# ax2 = ax1.twinx()
# ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
# ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# print("Best 20 cluster numbers: ", np.array(no_clusters0)[np.argsort(silhouette_scores0)[-20:]])
# print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.show()

from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters0, curve='convex', direction='decreasing')
print(kn.knee)
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=True)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1,min_dist=0.1, n_neighbors=1737, n_components=10, n_epochs=10000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
# silhouette_scores0 = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores0.append(davies_bouldin_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# ax2 = ax1.twinx()
# ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
# ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# print("Best 20 cluster numbers: ", np.array(no_clusters0)[np.argsort(silhouette_scores0)[-20:]])
# print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.show()

from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters0, curve='convex', direction='decreasing')
print(kn.knee)
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=True)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=2,min_dist=0.1, n_neighbors=1737, n_components=10, n_epochs=10000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
# silhouette_scores0 = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores0.append(davies_bouldin_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# ax2 = ax1.twinx()
# ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
# ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# print("Best 20 cluster numbers: ", np.array(no_clusters0)[np.argsort(silhouette_scores0)[-20:]])
# print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.show()

from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters0, curve='convex', direction='decreasing')
print(kn.knee)
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=True)

In [ ]:

from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters11, curve='convex', direction='decreasing')
print(kn.knee)
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=True)

In [ ]:
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=True)

In [ ]:
split_this = sch.fcluster(linkage, kn.knee, criterion='distance')
clusters52_split = np.split(np.argsort(split_this), indices_or_sections=np.cumsum(np.unique(split_this, return_counts=True)[1][:-1]))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=2,min_dist=0.1, n_neighbors=1737, n_components=30, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
# silhouette_scores0 = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores0.append(davies_bouldin_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# ax2 = ax1.twinx()
# ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
# ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# print("Best 20 cluster numbers: ", np.array(no_clusters0)[np.argsort(silhouette_scores0)[-20:]])
# print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.show()

from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters0, curve='convex', direction='decreasing')
print(kn.knee)
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=True)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)
pca_pseudo_CLR_transformed = pca.transform(pseudo_CLR_transformed)
print("PCA components: ",pca.explained_variance_ratio_.shape)
UMAP_reducer = umap.UMAP(random_state=1)#2,min_dist=0.1, n_neighbors=1737, n_components=30, n_epochs=1000, transform_queue_size=10)
UMAP_reducer.fit(pca_pseudo_CLR_transformed)
UMAP_embedding = UMAP_reducer.transform(pca_pseudo_CLR_transformed)

plt.rcParams['figure.figsize'] = [5,5]
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(UMAP_embedding[:,0], UMAP_embedding[:,1], c="black", s=s)
plt.show()


data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("distance", size=6, x=1)
plt.show()

from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores0 = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores0.append(silhouette_score(data, clusters))#, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [5,5]
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters0)[np.argsort(silhouette_scores0)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.show()

from kneed import KneeLocator
kn = KneeLocator(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters0, curve='convex', direction='decreasing')
print(kn.knee)
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=True)

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
# no_clusters = []
# silhouette_scores0 = []
# for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
#     clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores0.append(silhouette_score(data, clusters))#, metric=metric, random_state=0))
#     no_clusters.append(len(np.unique(clusters)))

fig, ax1 = plt.subplots()
plt.rcParams['figure.figsize'] = [3.32,3.32]
ax1.axvline(8.133220147326256, c="cyan")
ax1.axhline(29, c="cyan")

ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
# ax2 = ax1.twinx()
# ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores0, c="cyan")
ax1.set_ylabel("No. clusters")
# ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
print("Best 20 cluster numbers: ", np.array(no_clusters0)[np.argsort(silhouette_scores0)[-20:]])
print("Best 20 distances: ", np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1])[np.argsort(silhouette_scores0)[-20:]])
plt.savefig('figures/agglomerative_knee.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()


In [ ]:
data = UMAP_embedding
metric = "euclidean"
method="ward"
linkage = sch.linkage(data, method=method, metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0, color_threshold=8.133220147326256)
axis = plt.gca()
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Distance threshold", size=6, x=1)
axis.set_xlabel("Leaves of the dendogram", size=6)
plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance')).shape

In [ ]:
np.unique(clusters29)

In [ ]:
clusters29 = sch.fcluster(linkage, kn.knee, criterion='distance')-1
clusters29_split = np.split(np.argsort(clusters31), indices_or_sections=np.cumsum(np.unique(clusters31, return_counts=True)[1][:-1]))

plt.rcParams['figure.figsize'] = [3.32,3.32]

print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
# plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
for cluster in clusters29_split:
    plt.scatter(UMAP_embedding[:,0][cluster], UMAP_embedding[:,1][cluster], s=1)
plt.xlabel("UMAP axis 1")
plt.ylabel("UMAP axis 2")

plt.savefig('figures/UMAP_agglomerative.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()
n+=1

In [ ]:
pseudo_CLR_transformed.index.values[0] in ob_state.keys()

In [ ]:
ob_state[pseudo_CLR_transformed.index.values[0]]

In [ ]:
new_classification = clusters29

Belloni_classes = np.array(["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega", "Unknown"])

comparison_matrix = np.zeros((len(np.unique(new_classification)), len(Belloni_classes)), dtype=int)

comparison_matrix_df = pd.DataFrame(comparison_matrix, columns=Belloni_classes, index=np.unique(new_classification))

for n_Bc, Belloni_class in enumerate(Belloni_classes):
    if 
    Belloni_class_indices = np.where(np.array(segment_class) == Belloni_class)[0]
    count_clusters_for_class = np.unique(np.take(new_classification, Belloni_class_indices), return_counts=True)
    for cluster_ind, cluster in enumerate(count_clusters_for_class[0]):
        comparison_matrix_df[Belloni_class][cluster] = count_clusters_for_class[1][cluster_ind]

In [ ]:
class_normalized_comparison_matrix_df=(comparison_matrix_df-comparison_matrix_df.min())/(comparison_matrix_df.max()-comparison_matrix_df.min())
known_comparison_matrix_df = comparison_matrix_df.drop(columns=['Unknown']).T
component_normalized_comparison_matrix_df = (known_comparison_matrix_df-known_comparison_matrix_df.min())/(known_comparison_matrix_df.max()-known_comparison_matrix_df.min())

In [ ]:
# find particularly class-homogeneous components

good_comps = {}
for comp in range(122):
    comp_counts = comparison_matrix_df.iloc[comp,:].sort_values(ascending=False)
    comp_counts_nonzero = comp_counts.where(comp_counts>0).dropna().astype(int)
    if comp_counts_nonzero.index[0] == "Unknown":
        comp_class_proportion = comp_counts_nonzero[1:]/comp_counts_nonzero[1:].sum()
        if comp_class_proportion[0]>0.999:
            good_comps[comp] = comp_counts_nonzero
    else:
        good_comps[comp] = comp_counts_nonzero
print(len(good_comps))
# dominated_comps = []
# for i,v in good_comps.items():
#     if v.index[1] == "chi":
#         dominated_comps.append(i)
# print(dominated_comps)

In [ ]:
# chi is >95% bar unknowns : (10), 3, 5, 7, 15, 24, 28, 37, 44, 53, 57, 60, 69, 76, 91, 103, 112
# kappa : (79),  4, 42, 78, 104 (87 has 90.8% kappa)
# theta : 25, 26, 71
# lambda : 66
# rho : 12, 18, 19, 20, 22, 23, 30, 38, 45, 49, 50, 54, 56, 63, 65, 74, 77, 86, 97, 107

print(good_comps)

In [ ]:
comparison_matrix_df.iloc[98]#/(comparison_matrix_df.iloc[87].sum()-2628)

In [ ]:
comp_list = []
for i,v in good_comps.items():
    if len(v)>1:
        comp_list.append(i)

In [ ]:
print(comp_list)

In [ ]:
comp_counts = comparison_matrix_df.iloc[7,:].sort_values(ascending=False)
comp_counts_nonzero = comp_counts.where(comp_counts>0).dropna().astype(int)
comp_counts_nonzero[1:]/comp_counts_nonzero[1:].sum()

In [ ]:
comparison_matrix_df.sum(axis=0)

In [ ]:
comparison_matrix_df.T.iloc[:,66]

In [ ]:
(0.002524/1.002524)*7617

In [ ]:
np.unique(shape_moments_GM114_labels, return_counts=True)[1][10]

In [ ]:
np.where(comparison_matrix_df.T.idxmax().values != "Unknown")

In [ ]:
plt.rcParams['figure.figsize'] = (6.97, 6.97*(1/3))
import seaborn as sns

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)



ax = sns.heatmap(class_normalized_comparison_matrix_df.T, xticklabels=True, yticklabels=True, cmap='coolwarm')#, linewidth=0.5)
# # ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 6)
# ax.tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="out")

ax.xaxis.set_major_locator(MultipleLocator(4))
ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))

# For the minor ticks, use no labels; default NullFormatter.
# ax.xaxis.set_minor_locator(MultipleLocator(1.5))
ax.xaxis.set_minor_locator(AutoMinorLocator())


# for label in ax.xaxis.get_majorticklabels():
#     label.set_transform(label.get_transform() + 0.5)
    
# for label in ax.xaxis.get_minorticklabels():
#     label.set_transform(label.get_transform() + 0.5)

# plt.title("Gaussian mixture components' populations in terms of classified data (component-wise min-maxed)")
plt.savefig("figures/GMM122vsBelloni_heatmap.png", dpi=300)
plt.show()

In [ ]:
np.unique(sch.fcluster(linkage, kn.knee, criterion='distance'), return_counts=False)

In [ ]:
len(clusters53_split)

In [ ]:
index_1738 = [id_index for id_index, id_ in enumerate(ids) if id_ in pseudo_CLR_transformed.index.values]
lcs_1738 = [lcs[index] for index in index_1738]

In [ ]:
clusters53 = sch.fcluster(linkage, kn.knee, criterion='distance')
clusters53_split = np.split(np.argsort(clusters53), indices_or_sections=np.cumsum(np.unique(clusters53, return_counts=True)[1][:-1]))

In [ ]:
lc_start_times = []
lc_end_times = []
for lc in lcs_1738:
    lc_start_times.append(lc[0][0])
    lc_end_times.append(lc[0][-1])

In [ ]:
(np.sort(lc_end_times) - np.sort(lc_start_times))

In [ ]:
pseudo_CLR_transformed.index.values

In [ ]:
gap_duration = (np.sort(lc_start_times)[1:] - np.sort(lc_end_times)[:-1])

In [ ]:
cluster_labels_chronologically = np.take(clusters53, np.argsort(lc_start_times))

In [ ]:
plt.rcParams['figure.figsize'] = (20, 3)

plt.scatter(np.take(lc_start_times, np.argsort(lc_start_times)), cluster_labels_chronologically)
plt.show()

In [ ]:
len(transitions)

In [ ]:
gap_trans

In [ ]:
self_trans

In [ ]:
transitions=[]
self_trans = 0
gap_trans = 0
for gap_ind, gap in enumerate(gap_duration):
    if gap>10e3:
        gap_trans+=1
    if cluster_labels_chronologically[gap_ind]== cluster_labels_chronologically[gap_ind+1]:
        self_trans+=1
    if gap<10e3:# and cluster_labels_chronologically[gap_ind]!= cluster_labels_chronologically[gap_ind+1]:
        transitions.append((cluster_labels_chronologically[gap_ind], cluster_labels_chronologically[gap_ind+1]))

In [ ]:
transitions_df = pd.DataFrame(np.zeros((53,53)))
for trans in transitions:
    transitions_df.iloc[trans[0]-1, trans[1]-1] +=1
    
# transitions_df.columns = np.array(range(53))+1
# transitions_df.index = np.array(range(53))+1

In [ ]:
transitions_df

In [ ]:
def cmap_discretize(cmap, N):
    """Return a discrete colormap from the continuous colormap cmap.

        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.

    Example
        x = resize(arange(100), (5,100))
        djet = cmap_discretize(cm.jet, 5)
        imshow(x, cmap=djet)
    """

    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = concatenate((linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = linspace(0, 1., N+1)
    cdict = {}
    for ki,key in enumerate(('red','green','blue')):
        cdict[key] = [ (indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki]) for i in xrange(N+1) ]
    # Return colormap object.
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)

In [ ]:
plt.rcParams['figure.figsize'] = [3.32,3.32]
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
# trans_heatmap= 

# cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))
cmap = sns.color_palette("rocket", n_colors=5)# cmap_discretize(cmap, N)
cmap_new = []
cmap_new.append((cmap[0][0]/4,cmap[0][1]/4,cmap[0][2]/4))
for c in cmap[1:]:
    cmap_new.append(c)
# cmap = [cmap[0],cmap[2], cmap[3], cmap[4],cmap[5]]
sns.heatmap(transitions_df, cmap=cmap_new)
axes=plt.gca()
axes.set_xticks(np.array(range(0,53,3))+0.5)
axes.set_xticklabels(list(range(1,54,3)))
axes.set_yticks(np.array(range(0,53,3))+0.5)
axes.set_yticklabels(list(range(1,54,3)))
axes.set_ylabel("Initial cluster index")
axes.set_xlabel("Final cluster index")

colorbar = axes.collections[0].colorbar
colorbar.set_ticks(np.array(range(5))*4/5+2/5)
colorbar.set_ticklabels([0,1,2,3,4])

# axes[1][1].set_xticks([500,1000,1500,2000])

# trans_heatmap.pivot("Cluster index", "Cluster index", "No. transitions")
plt.savefig('figures/transitions_heatmap.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
pd.Series(np.diag(transitions_df), index=[transitions_df.index, transitions_df.columns])

##### transitions_df.iloc[np.diag_indices(53)]

In [ ]:
plt.rcParams['figure.figsize'] = [3.32,3.32]
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
# trans_heatmap= 

# cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))
cmap = sns.color_palette("rocket", n_colors=5)# cmap_discretize(cmap, N)
cmap_new = []
cmap_new.append((cmap[0][0]/4,cmap[0][1]/4,cmap[0][2]/4))
for c in cmap[1:]:
    cmap_new.append(c)
# cmap = [cmap[0],cmap[2], cmap[3], cmap[4],cmap[5]]
sns.heatmap(transitions_df.T, cmap=cmap_new)
axes=plt.gca()
axes.set_xticks(np.array(range(0,53,3))+0.5)
axes.set_xticklabels(list(range(0,53,3)))
axes.set_yticks(np.array(range(0,53,3))+0.5)
axes.set_yticklabels(list(range(0,53,3)))
axes.set_ylabel("Initial cluster index")
axes.set_xlabel("Final cluster index")

colorbar = axes.collections[0].colorbar
colorbar.set_ticks(np.array(range(5))*4/5+2/5)
colorbar.set_ticklabels([0,1,2,3,4])

# axes[1][1].set_xticks([500,1000,1500,2000])

# trans_heatmap.pivot("Cluster index", "Cluster index", "No. transitions")
# plt.savefig('figures/transitions_heatmap.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
transitions_df

In [ ]:
cmap_new = []
cmap_new.append((cmap[0][0]/2,cmap[0][1]/2,cmap[0][2]/2))
for c in cmap[1:]:
    cmap_new.append(c)

In [ ]:
np.triu_indices(53, k=1)[0].shape[0]

In [ ]:
rand_counts = np.zeros(1378*2)
rand_counts[np.random.choice(np.array(range(1378*2)), size=682)]+=1

In [ ]:
res = np.zeros(100000)
for i in range(100000):
    rand_counts = np.zeros(1378*2)
    rand_counts[np.random.choice(np.array(range(1378*2)), size=682)]+=1
    res[i] = np.sqrt(((rand_counts[:1378] - rand_counts[1378:])**2).mean())
    print(i)
    clear_output(wait=True)

In [ ]:
triu_vals = np.array(transitions_df)[np.triu_indices(53, k=1)]

In [ ]:
tril_vals = np.array(transitions_df).T[np.triu_indices(53, k=1)]

In [ ]:
np.sqrt(((triu_vals-tril_vals)**2).mean())

In [ ]:
from scipy.stats import ttest_1samp
ttest_1samp(res, 0.58)

In [ ]:
print(np.mean(res), np.std(res))

In [ ]:
(np.sqrt(((triu_vals-tril_vals)**2).mean())-np.mean(res))/np.std(res)

In [ ]:
plt.hist(res, bins=20)
ax = plt.gca()
ax.axvline(np.sqrt(((triu_vals-tril_vals)**2).mean()), c="magenta")
plt.show()

In [ ]:
np.triu_indices(53)
np.tril_indices(53)

In [ ]:
x = np.random.shuffle(x)
print(x)

In [ ]:
x=x*682/x.sum()
x

In [ ]:
x=np.random.rand(53,53)
x[np.diag_indices(53)]=0

In [ ]:
((transitions_df-transitions_df.T)**2).mean()

In [ ]:
import networkx as nx

G=nx.Graph()
for node in range(53):
    G.add_node(str(node+1))

for trans in transitions:
    G.add_edge(str(trans[0]), str(trans[1]))

In [ ]:
nx.draw(G)
# plt.savefig('figures/transitions_graph.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
plt.scatter(lc_start_times, np.ones(len(lc_start_times)), s=0.1)
plt.show()

In [ ]:
lc_start_times

In [ ]:
plt.rcParams['figure.figsize'] = (20, 3)

# which_to_plot = np.flip(np.take(pseudo_CLR_transformed.index.values, np.argsort(embedding[:,0])[-271:]))
which_to_plot = np.take(pseudo_CLR_transformed.index.values,clusters50_split[23])

for ob_ind, ob in enumerate(which_to_plot):
#     if np.sort(ob)[-1] > 0.9:
#     if np.argsort(ob)[-1] == 22:
        lc_index = np.where(np.array(ids) == ob)[0][0]
        fig, ax = plt.subplots(nrows=1, ncols=2)
        ax[0].plot(lcs[lc_index][0], lcs[lc_index][1])
        ax[1].plot(lcs[lc_index][0]-lcs[lc_index][0][0], lcs[lc_index][1])
        ax[1].set_ylim((0,10000))
        ax[1].set_xlim((0,3000))

        plt.show()
#         print(list(ObID_GaussComps_dict_122_chrono.keys())[ob_ind], np.argsort(ob)[-5:], np.sort(ob)[-5:])
#     print(np.argsort(ob), np.sort(ob)[-3:])

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)

# print(pca.explained_variance_ratio_.shape)

reducer = umap.UMAP(random_state=1, n_neighbors=50, n_components=2)
reducer.fit(pca.transform(pseudo_CLR_transformed))
embedding = reducer.transform(pca.transform(pseudo_CLR_transformed))

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]

print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:

# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")
data = embedding#pca.transform(pseudo_CLR_transformed)
metric = "euclidean"

linkage = sch.linkage(data, method="complete", metric=metric)
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("euclidean distance", size=6, x=1)Connectivity matrices

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]

no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(0.98298298, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)
print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])
plt.show()

In [ ]:
skip_first=1
skip_last=400

np.array(np.linspace(y_bottom_limit,y_top_limit,1000)[skip_first:-1-skip_last])[np.argsort(silhouette_scores[skip_first:-skip_last])[-20:]]

In [ ]:
np.unique( sch.fcluster(linkage, 2.31247118, criterion='distance'), return_counts=1)

In [ ]:
clusters13 = sch.fcluster(linkage, 2.31247118, criterion='distance')

In [ ]:
split_this = clusters13
clusters13_split = np.split(np.argsort(split_this), indices_or_sections=np.cumsum(np.unique(split_this, return_counts=True)[1][:-1]))

In [ ]:
n=0

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]

print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
for cluster in clusters40_split[n]:
    plt.scatter(embedding[:,0][cluster], embedding[:,1][cluster], c="cyan", s=s)
plt.show()
print(n)
n+=1

In [ ]:
clusters9_split

In [ ]:
print("UMAP of CLR transformed data, PCA (65PCs, 0.95 variance), Euclidean distance")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
from sklearn.manifold import Isomap
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import SpectralEmbedding
from sklearn.manifold import MDS
from sklearn.manifold import TSNE





isomap_mapper = TSNE(n_components=2)#, eigen_solver="dense")
X_isomap_transformed = isomap_mapper.fit_transform(pseudo_CLR_transformed)

In [ ]:
plt.rcParams['figure.figsize'] = (5, 5)

plt.scatter(X_isomap_transformed[:,0], X_isomap_transformed[:,1], c="black", s=s)
plt.show()

In [ ]:
x_lim = 7.5
len(np.take(pseudo_CLR_transformed.index.values, np.where(embedding[:,0]>x_lim)[0]))
np.flip(np.argsort(embedding[:,0])[-271:])

In [ ]:
with open('{}/1776_light_curves_1s_bin_errorfix.pkl'.format(data_dir), 'rb') as f:
    lcs = pickle.load(f)
with open('{}/1776_light_curves_1s_bin_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    ids = pickle.load(f)

In [ ]:
clusters13_split[3].shape

In [ ]:
plt.rcParams['figure.figsize'] = (20, 3)

# which_to_plot = np.flip(np.take(pseudo_CLR_transformed.index.values, np.argsort(embedding[:,0])[-271:]))
which_to_plot = np.take(pseudo_CLR_transformed.index.values,clusters13_split[1])

for ob_ind, ob in enumerate(which_to_plot):
#     if np.sort(ob)[-1] > 0.9:
#     if np.argsort(ob)[-1] == 22:
        lc_index = np.where(np.array(ids) == ob)[0][0]
        fig, ax = plt.subplots(nrows=1, ncols=2)
        ax[0].plot(lcs[lc_index][0], lcs[lc_index][1])
        ax[1].plot(lcs[lc_index][0]-lcs[lc_index][0][0], lcs[lc_index][1])
        ax[1].set_ylim((0,10000))
        ax[1].set_xlim((0,3000))

        plt.show()
#         print(list(ObID_GaussComps_dict_122_chrono.keys())[ob_ind], np.argsort(ob)[-5:], np.sort(ob)[-5:])
#     print(np.argsort(ob), np.sort(ob)[-3:])

In [ ]:
lc_index

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(pseudo_CLR_transformed)

print(pca.explained_variance_ratio_.shape)


In [ ]:
pca.explained_variance_ratio_

In [ ]:
np.max(normalized_obs_component_counts_df_122.iloc[:,120].values)

In [ ]:
zscore(normalized_obs_component_counts_df_122, axis=0)

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(zscore(normalized_obs_component_counts_df_122, axis=0))


embedding = reducer.transform(zscore(normalized_obs_component_counts_df_122, axis=0))


In [ ]:
print("normalised row, zscored column")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(normalized_obs_component_counts_df_122)


embedding = reducer.transform(normalized_obs_component_counts_df_122)


In [ ]:
print("normalised rows")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")
reducer = umap.UMAP(random_state=42)
reducer.fit(row_ratio)


embedding = reducer.transform(row_ratio)


In [ ]:
print("div row by sum")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(obs_component_counts_df_122)


embedding = reducer.transform(obs_component_counts_df_122)


In [ ]:
print("raw")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")
reducer = umap.UMAP(random_state=42, metric="manhattan")
reducer.fit(row_ratio)


embedding = reducer.transform(row_ratio)


In [ ]:
print("div row by sum, manhattan")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")
reducer = umap.UMAP(random_state=42, metric="cosine")
reducer.fit(row_ratio)


embedding = reducer.transform(row_ratio)


In [ ]:
print("div row by sum, cosine")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()
chebyshev

In [ ]:
row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")
reducer = umap.UMAP(random_state=42, metric="chebyshev")
reducer.fit(row_ratio)


embedding = reducer.transform(row_ratio)


In [ ]:
print("div row by sum, chebyshev")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()
chebyshev

In [ ]:
row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")
reducer = umap.UMAP(random_state=42, metric="braycurtis")
reducer.fit(row_ratio)


embedding = reducer.transform(row_ratio)
#. Furthermore, principal-component analysis (PCA) has common assumptions of normally distributed and linearly related variables, often violated by biological data (4–7). As a result, classical distance metrics that take into account only the presence/absence of taxa, such as the Jaccard index, or metrics that explicitly account for relative abundances, such as Bray-Curtis symmetrized distance, are commonly used

In [ ]:
plt.rcParams['figure.figsize'] = (3.32, 3.32)
print("div row by sum, braycurtis")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")
reducer = umap.UMAP(random_state=42, metric="jaccard")
reducer.fit(row_ratio)


embedding = reducer.transform(row_ratio)


In [ ]:
plt.rcParams['figure.figsize'] = (3.32, 3.32)
print("div row by sum, jaccard")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
reducer = umap.UMAP(random_state=42, metric="euclidean")
reducer.fit(non_zero_row_ratio)


embedding = reducer.transform(non_zero_row_ratio)


In [ ]:
print("x*1000+1 div row by sum, CLR, aitchison/ euclidean")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
row_ratio

In [ ]:
row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

reducer = umap.UMAP(random_state=42, metric="ll_dirichlet")
reducer.fit(non_zero_row_ratio)


embedding = reducer.transform(non_zero_row_ratio)


In [ ]:
print("div row by sum, mahalanobis")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

# Perform agglomerative clustering of observations in Gaussian cluster space

In [ ]:
from sklearn import cluster
agglo = cluster.FeatureAgglomeration()
agglo.fit(normalized_obs_component_counts_df_122)

In [ ]:
transformed_features = agglo.transform(normalized_obs_component_counts_df_122)

In [ ]:
def _validate_aitchison(w, u_v):
    if np.any(w <= 0):
        raise ValueError('Values in %s should be positive.' % u_v)
    return(w)

def _validate_vector(u, dtype=None):
    # XXX Is order='c' really necessary?
    u = np.asarray(u, dtype=dtype, order='c').squeeze()
    # Ensure values such as u=1 and u=[1] still return 1-D arrays.
    u = np.atleast_1d(u)
    if u.ndim > 1:
        raise ValueError("Input vector should be 1-D.")
    return u

def aitchison(u, v):
    """
    Computes the Aitchison distance between two 1-D arrays.
    The Aitchison distance between 1-D arrays `u` and `v`, is defined as
    .. math::
       {||log \left( \frac{u}{v} \right) - mean \left(log \left( \frac{u}{v} \right) \right) ||}_2
       
    Parameters
    ----------
    u : (N,) array_like
        Input array.
    v : (N,) array_like
        Input array.
    Returns
    -------
    aitchison : double
        The Aitchison distance between vectors `u` and `v`.
        
    Since Aitchison distance is just the Euclidean distance of the clr (center-log ratio) transformed data points, do you need a separate C-level function? Doing the transformations on a per-vector basis will be slower than precomputing them. Unless memory is a concern? This is somewhat similar to how cosine cdist is currently implemented.
    """
    
    u = _validate_vector(u)
    v = _validate_vector(v)
    _validate_aitchison(u, 'u')
    _validate_aitchison(v, 'v')
    log_u_v = np.log(u / v)
    dist = np.linalg.norm(log_u_v - np.mean(log_u_v), ord=2)
    return(dist)

In [ ]:
row_ratio.min()<0

In [ ]:
# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(obs_component_counts_df_122, method="complete", metric="cosine")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0, color_threshold=0.98298298)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Cosine distance", size=6, x=1)

plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
np.array(no_clusters[-200:])[np.argsort(calinski_harabasz_scores[-200:])]#[-100:]

In [ ]:
np.array(no_clusters)[np.argsort(silhouette_scores)][-100:]

In [ ]:
np.vstack((np.array(silhouette_scores),np.array(no_clusters))).T[-50:]

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


# y_limit = linkage[-1][2]
# no_clusters = []
# silhouette_scores = []
# # calinski_harabasz_scores = []
# for distance in np.linspace(0,y_limit,1000)[:-1]:
#     clusters = sch.fcluster(linkage, distance, criterion='distance')
#     silhouette_scores.append(silhouette_score(obs_component_counts_df_122, clusters, metric="cosine", random_state=0))
#     no_clusters.append(len(np.unique(clusters)))
# #     calinski_harabasz_scores.append(davies_bouldin_score(row_ratio, clusters))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
ax1.axvline(0.98298298, c="black")
ax1.axhline(60, c="black")
ax1.plot(np.linspace(0,y_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(0,y_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
# ax3.set_ylabel("davies_bouldin_score")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

In [ ]:

# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(obs_component_counts_df_122, method="complete", metric="braycurtis")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0, color_threshold=0.98298298)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Cosine distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
np.unique(clusters).shape

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))


y_limit = linkage[-1][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(0.01,y_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(obs_component_counts_df_122, clusters, metric="braycurtis", random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
ax1.axvline(0.98298298, c="black")
ax1.axhline(60, c="black")
ax1.plot(np.linspace(0.01,y_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(0.01,y_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

In [ ]:
from scipy.stats.mstats import gmean

In [ ]:
np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1]

In [ ]:
plt.close()

plt.rcParams['figure.figsize'] = (30,30)
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(nrows=5, ncols=5)

tril = np.array(np.tril_indices(5)).T

data = obs_component_counts_df_122.div(np.sum(obs_component_counts_df_122, axis=1), axis="rows")

for x,y in tril:
    axes[x,y].scatter(data.iloc[:,x],data.iloc[:,y])
    axes[x,y].set_ylim((-0.1,1))
    axes[x,y].set_xlim((-0.1,1))
    axes[x,y].text(0.99, 0.99, np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1], ha='right', va='top', transform=axes[x,y].transAxes, size=10)

plt.show()


In [ ]:
plt.close()

plt.rcParams['figure.figsize'] = (30,30)
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(nrows=10, ncols=10)

tril = np.array(np.tril_indices(10)).T

data = pseudo_CLR_transformed

for x,y in tril:
    axes[x,y].scatter(data.iloc[:,x],data.iloc[:,y])
#     axes[x,y].set_ylim((-0.1,1))
#     axes[x,y].set_xlim((-0.1,1))
    axes[x,y].text(0.99, 0.99, np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1], ha='right', va='top', transform=axes[x,y].transAxes, size=10)

plt.show()


In [ ]:
pseudo_counts = (obs_component_counts_df_122*2).replace(to_replace=0, value=1)
closed_pseudo_counts = pseudo_counts.div(np.sum(pseudo_counts, axis=1), axis="rows")

scaled_pseudo_counts = closed_pseudo_counts.div(gmean(closed_pseudo_counts, axis=1), axis="rows")
pseudo_CLR_transformed = np.log(scaled_pseudo_counts)

In [ ]:
scaled_pseudo_counts

In [ ]:
plt.close()

plt.rcParams['figure.figsize'] = (30,30)
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(nrows=10, ncols=10)

tril = np.array(np.tril_indices(10)).T

data = pseudo_CLR_transformed

for x,y in tril:
    axes[x,y].scatter(data.iloc[:,x],data.iloc[:,y])
#     axes[x,y].set_ylim((-0.1,1))
#     axes[x,y].set_xlim((-0.1,1))
    axes[x,y].text(0.99, 0.99, np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1], ha='right', va='top', transform=axes[x,y].transAxes, size=10)

plt.show()


In [ ]:
pseudo_counts = (obs_component_counts_df_122*1e1).replace(to_replace=0, value=1)
closed_pseudo_counts = pseudo_counts.div(np.sum(pseudo_counts, axis=1), axis="rows")

scaled_pseudo_counts = closed_pseudo_counts.div(gmean(closed_pseudo_counts, axis=1), axis="rows")
pseudo_CLR_transformed = np.log(scaled_pseudo_counts)

In [ ]:
pseudo_CLR_transformed

In [ ]:
obs_component_counts_df_122.div(np.mean(obs_component_counts_df_122, axis=1).values, axis="rows")

In [ ]:
np.mean(obs_component_counts_df_122, axis=1).values

In [ ]:
pseudo_CLR_transformed

In [ ]:

# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(pseudo_CLR_transformed, method="complete", metric="euclidean")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Cosine distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))

data = pseudo_CLR_transformed
metric = "euclidean"

y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(0.98298298, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit, y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])

In [ ]:
2**122

In [ ]:

# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(pseudo_CLR_transformed, method="complete", metric="cosine")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Cosine distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))

data = pseudo_CLR_transformed
metric = "cosine"

y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(0.98298298, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit, y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# ax1.set_ylim((0,200))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])

In [ ]:
pseudo_counts = (obs_component_counts_df_122*1000).replace(to_replace=0, value=1)
scaled_pseudo_counts = pseudo_counts.div(gmean(pseudo_counts, axis=1), axis="rows")
pseudo_CLR_transformed_1000 = np.log(scaled_pseudo_counts)

In [ ]:
obs_component_counts_df_122.div(np.mean(obs_component_counts_df_122, axis=1).values, axis="rows")

In [ ]:
pseudo_CLR_transformed

In [ ]:

# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(pseudo_CLR_transformed_1000, method="complete", metric="euclidean")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Cosine distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))

data = pseudo_CLR_transformed_1000
metric = "euclidean"

y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(0.98298298, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit, y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# plt.ylim((0,50))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])

In [ ]:

# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(pseudo_CLR_transformed_1000, method="complete", metric="cosine")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Cosine distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))

data = pseudo_CLR_transformed_1000
metric = "cosine"

y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(0.98298298, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit, y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# ax1.set_ylim((0,200))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])

In [ ]:
reducer = umap.UMAP(random_state=42, metric="euclidean")
reducer.fit(pseudo_CLR_transformed_1000)


embedding = reducer.transform(pseudo_CLR_transformed_1000)


In [ ]:
print("CLR with pseudo counts/ euclidean")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
reducer = umap.UMAP(random_state=42, metric="euclidean")
reducer.fit(pseudo_CLR_transformed)


embedding = reducer.transform(pseudo_CLR_transformed_1000)


In [ ]:
print("CLR with pseudo counts/ euclidean")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
pseudo_counts = (obs_component_counts_df_122*1e5).replace(to_replace=0, value=1)
scaled_pseudo_counts = pseudo_counts.div(gmean(pseudo_counts, axis=1), axis="rows")
pseudo_CLR_transformed_1e5 = np.log(scaled_pseudo_counts)

In [ ]:
reducer = umap.UMAP(random_state=42, metric="euclidean")
reducer.fit(pseudo_CLR_transformed)


embedding = reducer.transform(pseudo_CLR_transformed_1e5)


In [ ]:
print("CLR with pseudo counts/ euclidean")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
pseudo_CLR_transformed

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99)
pca.fit(pseudo_CLR_transformed)

print(pca.explained_variance_ratio_)


In [ ]:
from sklearn.decomposition import TruncatedSVD
# from scipy.sparse import random as sparse_random
# from sklearn.random_projection import sparse_random_matrix
# X = sparse_random(100, 100, density=0.01, format='csr',
#                   random_state=42)
svd = TruncatedSVD(n_components=100,random_state=42)
svd.fit(pseudo_CLR_transformed)

print(svd.explained_variance_ratio_)

print(svd.explained_variance_ratio_.sum())

print(svd.singular_values_)

In [ ]:
pseudo_counts = pd.DataFrame(lda.transform(obs_component_counts_df_122))
scaled_pseudo_counts = pseudo_counts.div(gmean(pseudo_counts, axis=1), axis="rows")
pseudo_CLR_transformed_1e5 = np.log(scaled_pseudo_counts)

In [ ]:
reducer = umap.UMAP(random_state=42, metric="euclidean")
reducer.fit(pseudo_CLR_transformed_1e5)


embedding = reducer.transform(pseudo_CLR_transformed_1e5)


In [ ]:
plt.rcParams['figure.figsize'] = (5,5)

print("counds with dirchlet decomposition")
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
pseudo_CLR_transformed_1e5

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=122,random_state=0)
lda.fit(obs_component_counts_df_122)

# get topics for some given samples:
lda.transform(obs_component_counts_df_122)

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.plot(np.diff(pca.explained_variance_ratio_, n=2))
# plt.ylim((0,0.25))
plt.show()

In [ ]:
np.diff(pca.explained_variance_ratio_, n=2)

In [ ]:
np.sum(pca.explained_variance_ratio_[:20])

In [ ]:
np.where((np.cumsum(pca.explained_variance_ratio_)>0.99) == 1)[0]+1

In [ ]:
from sklearn.cluster import OPTICS

clustering = OPTICS(min_samples=5).fit(log_lda_components_probs)
clustering.labels_

In [ ]:
np.unique(clustering.labels_, return_counts=True)

In [ ]:
# # Reachability plot
# clust = optics_shape_moments

# plt.rcParams['figure.figsize'] = (100,10)
# plt.rcParams.update({'font.size': 42})



# # labels_050 = cluster_optics_dbscan(reachability=clust.reachability_,
# #                                    core_distances=clust.core_distances_,
# #                                    ordering=clust.ordering_, eps=0.5)
# # labels_200 = cluster_optics_dbscan(reachability=clust.reachability_,
# #                                    core_distances=clust.core_distances_,
# #                                    ordering=clust.ordering_, eps=2)

space = np.arange(len(log_lda_components_probs))
reachability = clustering.reachability_[clustering.ordering_]
labels = clustering.labels_[clustering.ordering_]

# # breaks = [0,21430,199990,242305,303065,309196,371590,383575,391430,395315, 414530, 468201]#371590
# # for nb, brejk in enumerate(breaks[:-1]):
# #     Xk = space[labels == 0][brejk:breaks[nb+1]]
# #     Rk = reachability[labels == 0][brejk:breaks[nb+1]]
# #     plt.scatter(Xk, Rk, alpha=1)
    

# plt.plot(space[labels == -1], reachability[labels == -1], 'white', alpha=0.3)
plt.scatter(space, reachability, s=0.1)#, c=np.array(shape_moments_GM114_labels_volume_fake)[clust.ordering_], cmap="cool")
# plt.plot(space, np.full_like(space, 0.5, dtype=float), 'k-.', alpha=0.5)
plt.ylabel('Reachability (epsilon distance)')
# plt.set_title('Reachability Plot')
plt.show()

In [ ]:
np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]]

In [ ]:
np.argmin(silhouette_scores)#[-10:]

In [ ]:
silhouette_scores

# Try Latent Dirichlet Alocation on the observation x pattern matrix

In [ ]:
import gensim

In [ ]:
GaussComps_str = [[u'{}'.format(val) for val in ob] for ob in ObID_GaussComps_dict_122_chrono.values()]# if len(ob)>20]

In [ ]:
GaussComps_str[0]

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(GaussComps_str, min_count=100, threshold=0.25, scoring="npmi")#, min_count=2, threshold=1)#, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[GaussComps_str])#, min_count=2, threshold=1)#, threshold=100) 
quadgrams = gensim.models.Phrases(trigram[GaussComps_str])#, min_count=2, threshold=1)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
quadgrams_mod = gensim.models.phrases.Phraser(quadgrams)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def make_quadgrams(texts):
    return [quadgrams_mod[trigram_mod[bigram_mod[doc]]] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(GaussComps_str)
data_words_trigrams = make_trigrams(GaussComps_str)
data_words_quadgrams = make_quadgrams(GaussComps_str)

In [ ]:
# Create Dictionary
id2word = gensim.corpora.Dictionary(GaussComps_str)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in GaussComps_str]

# View
print(corpus[:1])

In [ ]:
# Create Dictionary
id2word_quad = gensim.corpora.Dictionary(data_words_quadgrams)

# Term Document Frequency
corpus_quad = [id2word_quad.doc2bow(text, return_missing=True) for text in data_words_quadgrams]

# View
print(corpus_quad[:1])

In [ ]:
# corpus_matrix = gensim.matutils.corpus2csc(corpus_quad[0])

In [ ]:
print(len(np.unique([item for sublist in data_words_quadgrams for item in sublist], return_counts=True)[0]))
np.unique([item for sublist in data_words_quadgrams for item in sublist], return_counts=True)

In [ ]:
gensim_lda_model = gensim.models.LdaMulticore(corpus=corpus_quad,
                                       id2word=id2word_quad,
                                       num_topics=1000, 
                                       random_state=0,
                                       chunksize=2000,
                                       passes=10,
                                       per_word_topics=True, 
                                       minimum_probability=0.01,
                                      workers=30)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import LatentDirichletAllocation

X = obs_component_counts_df_122
scoring = None#('perplexity', 'score')

results = []

for n_components in range(20):
    n_components+=1
    lda = LatentDirichletAllocation(n_components=n_components, random_state=0)
    cv_results = cross_validate(lda, X, cv=5, scoring=scoring, return_train_score=True)
    results.append((n_components, cv_results))
    print(n_components)
    clear_output(wait=True)

    
    n_components=10, *, doc_topic_prior=None, topic_word_prior=None, 
    learning_method='batch', learning_decay=0.7, learning_offset=10.0, max_iter=10, 
    batch_size=128, evaluate_every=-1, total_samples=1000000.0, perp_tol=0.1, mean_change_tol=0.001,
    max_doc_update_iter=100, n_jobs=None, verbose=0, random_state=None

In [ ]:
GaussComps_str_per_ob = [" ".join(np.array(ob, dtype=str)) for ob in ObID_GaussComps_dict_122_chrono.values()]

In [ ]:
GaussComps_str_per_ob

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
GaussComps_str_per_ob = [" ".join(np.array(ob, dtype=str)) for ob in ObID_GaussComps_dict_122_chrono.values()]
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 10), min_df=50)
X = vectorizer.fit_transform(GaussComps_str_per_ob)

In [ ]:
X

In [ ]:
vectorizer.vocabulary_

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=100, random_state=0, verbose=1, batch_size=2000, n_jobs=20, max_iter=1000)
lda.fit(X)

In [ ]:
lda.get_params()

In [ ]:
from scipy.stats.mstats import gmean
lda_components_probs = pd.DataFrame(lda.transform(X))
scaled_lda_components_probs = lda_components_probs.div(gmean(lda_components_probs, axis=1), axis="rows")
log_lda_components_probs = np.log(scaled_lda_components_probs)

In [ ]:
lda.transform(X).sum(axis=1).sum()

In [ ]:
pd.DataFrame(lda.transform(X))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
pca.fit(log_lda_components_probs)

print(pca.explained_variance_ratio_)

In [ ]:
len(pca.explained_variance_ratio_)

In [ ]:
pca.transform(log_lda_components_probs)

In [ ]:
plt.rcParams['figure.figsize'] = (5, 5)

plt.plot(pca.explained_variance_ratio_)
plt.show()

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(log_lda_components_probs)


embedding = reducer.transform(log_lda_components_probs)


In [ ]:
list(ObID_GaussComps_dict_122_chrono.keys())

In [ ]:
with open('{}/1776_light_curves_1s_bin_errorfix.pkl'.format(data_dir), 'rb') as f:
    lcs = pickle.load(f)
with open('{}/1776_light_curves_1s_bin_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    ids = pickle.load(f)

In [ ]:
np.where(np.array(ids) == "10408-01-03-00")[0][0]

In [ ]:
[2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 75, 76, 78, 79, 80, 81, 83, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 99]


In [ ]:
plt.rcParams['figure.figsize'] = (20, 3)

for ob_ind, ob in enumerate(lda.transform(X)):
#     if np.sort(ob)[-1] > 0.9:
    if np.argsort(ob)[-1] == 22:
        lc_index = np.where(np.array(ids) == list(ObID_GaussComps_dict_122_chrono.keys())[ob_ind])[0][0]
        fig, ax = plt.subplots(nrows=1, ncols=2)
        ax[0].plot(lcs[lc_index][0], lcs[lc_index][1])
        ax[1].plot(lcs[lc_index][0]-lcs[lc_index][0][0], lcs[lc_index][1])
        ax[1].set_ylim((0,10000))
        ax[1].set_xlim((0,3000))

        plt.show()
        print(list(ObID_GaussComps_dict_122_chrono.keys())[ob_ind], np.argsort(ob)[-5:], np.sort(ob)[-5:])
#     print(np.argsort(ob), np.sort(ob)[-3:])

In [ ]:
plt.plot(lda.transform(X)[2])
plt.show()

In [ ]:
good_comps = 0
good_comps_list = []
for component in range(100):
    if np.mean(lda.components_[0]) != np.mean(lda.components_[component]):
        good_comps+=1
        good_comps_list.append(component)
print(good_comps_list)

In [ ]:
plt.plot(lda.components_[1])
plt.show()

In [ ]:
pca.transform(log_lda_components_probs)

In [ ]:
import scipy.cluster.hierarchy as sch
# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(pca.transform(log_lda_components_probs), method="complete", metric="euclidean")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))

data = log_lda_components_probs
metric = "euclidean"

y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(0.98298298, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit, y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# ax1.set_ylim((0,200))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])

In [ ]:
import scipy.cluster.hierarchy as sch
# row_ratio = obs_component_counts_df_122.div(obs_component_counts_df_122.sum(axis=1).values, axis="rows")

linkage = sch.linkage(log_lda_components_probs, method="complete", metric="euclidean")
plt.rcParams['figure.figsize'] = (6.97, 3)
plt.rcParams.update({'font.size': 6})

dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
axis = plt.gca()
# axis.tick_params(axis="x", which="major", length=0, width=0, labelsize=0, direction="in")
axis.set_xticks([])
axis.set_xticklabels([])
axis.set_ylabel("Euclidean distance", size=6, x=1)

# plt.savefig('figures/agglomerative_dendogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)


plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))

data = log_lda_components_probs
metric = "euclidean"

y_top_limit = linkage[-1][2]
y_bottom_limit = linkage[0][2]
no_clusters = []
silhouette_scores = []
for distance in np.linspace(y_bottom_limit,y_top_limit,1000)[:-1]:
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    silhouette_scores.append(silhouette_score(data, clusters, metric=metric, random_state=0))
    no_clusters.append(len(np.unique(clusters)))
    
    
fig, ax1 = plt.subplots()


plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
# ax1.axvline(0.98298298, c="black")
# ax1.axhline(60, c="black")
ax1.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")

ax2 = ax1.twinx()
ax2.plot(np.linspace(y_bottom_limit, y_top_limit,1000)[:-1], silhouette_scores, c="cyan")
# plt.title("No. of clusters as the function of distance for hierarchical clustering")
# ax3 = ax1.twinx()
# ax3.plot(np.linspace(0,y_limit,1000)[:-1], calinski_harabasz_scores, c="orange")
# ax4 = ax1.twinx()
# ax4.plot(np.linspace(0,y_limit,1000)[:-2], np.diff(silhouette_scores, 1), c="green")

ax1.set_ylabel("No. clusters", c="magenta")
ax2.set_ylabel("Silhouette score", c="cyan")
ax1.set_xlabel("Distance threshold")
# ax1.set_ylim((0,200))
# plt.savefig('figures/agglomerative_silhouette.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

print("Best 20 cluster numbers: ", np.array(no_clusters)[np.argsort(silhouette_scores)[-20:]])

In [ ]:
plt.plot(np.linspace(y_bottom_limit,y_top_limit,1000)[:-1], no_clusters, c="magenta")
plt.ylim((0,250))
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (5,5)

print("lda 2451 vocab log")
s=0.5
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (5,5)

print("lda 893 vocab log")
s=0.5
plt.scatter(embedding[:,0], embedding[:,1], c="black", s=s)
plt.show()

In [ ]:
vectorizer.vocabulary_

In [ ]:
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim
metric_coherence_gensim(measure='c_v', 
                        top_n=100, 
                        topic_word_distrib=lda.components_, 
                        dtm=X, 
                        vocab=np.array([x for x in vectorizer.vocabulary_.keys()]), 
                        texts=GaussComps_str)

Signature:
metric_coherence_gensim(
    measure,
    topic_word_distrib=None,
    gensim_model=None,
    vocab=None,
    dtm=None,
    gensim_corpus=None,
    texts=None,
    top_n=20,
    return_coh_model=False,
    return_mean=False,
    **kwargs,
)
Docstring:
Calculate model coherence using Gensim's
`CoherenceModel <https://radimrehurek.com/gensim/models/coherencemodel.html>`_. See also this `tutorial
<https://rare-technologies.com/what-is-topic-coherence/>`_.

Define which measure to use with parameter `measure`:

- ``'u_mass'``
- ``'c_v'``
- ``'c_uci'``
- ``'c_npmi'``

Provide a topic word distribution `topic_word_distrib` OR a Gensim model `gensim_model`
and the corpus' vocabulary as `vocab` OR pass a gensim corpus as `gensim_corpus`. `top_n` controls how many most
probable words per topic are selected.

If measure is ``'u_mass'``, a document-term-matrix `dtm` or `gensim_corpus` must be provided and `texts` can be
None. If any other measure than ``'u_mass'`` is used, tokenized input as `texts` must be provided as 2D list::

    [['some', 'text', ...],          # doc. 1
     ['some', 'more', ...],          # doc. 2
     ['another', 'document', ...]]   # doc. 3

If `return_coh_model` is True, the whole :class:`gensim.models.CoherenceModel` instance will be returned, otherwise:

- if `return_mean` is True, the mean coherence value will be returned
- if `return_mean` is False, a list of coherence values (for each topic) will be returned

Provided `kwargs` will be passed to :class:`gensim.models.CoherenceModel` or
:meth:`gensim.models.CoherenceModel.get_coherence_per_topic`.

.. note:: This function also supports models from `lda` and `sklearn` (by passing `topic_word_distrib`, `dtm` and
          `vocab`)!

:param measure: the coherence calculation type; one of the values listed above
:param topic_word_distrib: topic-word distribution; shape KxM, where K is number of topics, M is vocabulary size if
                           `gensim_model` is not given
:param gensim_model: a topic model from Gensim if `topic_word_distrib` is not given
:param vocab: vocabulary list/array if `gensim_corpus` is not given
:param dtm: document-term matrix of shape NxM with N documents and vocabulary size M  if `gensim_corpus` is not
            given
:param gensim_corpus: a Gensim corpus if `vocab` is not given
:param texts: list of tokenized documents; necessary if using a `measure` other than ``'u_mass'``
:param top_n: number of most probable words selected per topic
:param return_coh_model: if True, return :class:`gensim.models.CoherenceModel` as result
:param return_mean: if `return_coh_model` is False and `return_mean` is True, return mean coherence
:param kwargs: parameters passed to :class:`gensim.models.CoherenceModel` or
               :meth:`gensim.models.CoherenceModel.get_coherence_per_topic`
:return: if `return_coh_model` is True, return :class:`gensim.models.CoherenceModel` as result; otherwise if
         `return_mean` is True, mean of all coherence values, otherwise array of length K with coherence per
         topic
File:      /export/data/jakubok/software/anaconda3/envs/py37/lib/python3.7/site-packages/tmtoolkit/topicmod/evaluate.py
Type:      function


In [ ]:
log_lda_components_probs

In [ ]:
plt.close()

plt.rcParams['figure.figsize'] = (30,30)
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(nrows=10, ncols=10)

tril = np.array(np.tril_indices(10)).T

data = log_lda_components_probs

for x,y in tril:
    axes[x,y].scatter(data.iloc[:,x],data.iloc[:,y])
    axes[x,y].set_ylim((-3,13))
    axes[x,y].set_xlim((-3,13))
    axes[x,y].text(0.99, 0.99, np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1], ha='right', va='top', transform=axes[x,y].transAxes, size=10)

plt.show()


In [ ]:
plt.close()

plt.rcParams['figure.figsize'] = (30,30)
plt.rcParams.update({'font.size': 10})

fig, axes = plt.subplots(nrows=10, ncols=10)

tril = np.array(np.tril_indices(10)).T

data = log_lda_components_probs

for x,y in tril:
    axes[x,y].scatter(data.iloc[:,x],data.iloc[:,y])
#     axes[x,y].set_ylim((-0.1,1))
#     axes[x,y].set_xlim((-0.1,1))
    axes[x,y].text(0.99, 0.99, np.corrcoef(data.iloc[:,x],data.iloc[:,y])[0,1], ha='right', va='top', transform=axes[x,y].transAxes, size=10)

plt.show()


In [ ]:
plt.hist(lda.transform(X).flatten())
plt.show()

In [ ]:
# Build LDA model
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=20, 
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=100, 
                                       random_state=0,
                                       chunksize=2000,
                                       passes=10,
                                       per_word_topics=True, 
                                       minimum_probability=0.01,
                                      workers=30)

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus_quad,
                                       id2word=id2word_quad,
                                       num_topics=1000, 
                                       random_state=0,
                                       chunksize=2000,
                                       passes=10,
                                       per_word_topics=True, 
                                       minimum_probability=0.01,
                                      workers=30)

In [ ]:
lda_model

In [ ]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics(num_topics=-1, num_words=122))
doc_lda = lda_model[corpus]

In [ ]:
len(lda_model.print_topics(num_topics=-1, num_words=10))

In [ ]:
doc_lda

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=GaussComps_str, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus_quad))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=data_words_quadgrams, dictionary=id2word_quad, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# import re
# import numpy as np
# import pandas as pd
# from pprint import pprint

# # Gensim
# import gensim
# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from gensim.models import CoherenceModel

# # spacy for lemmatization
# import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# # Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
def compute_coherence_values(id2word, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    perpexities=[]
    for num_topics in range(start, limit, step):
        print(num_topics)
        model = gensim.models.LdaMulticore(corpus=corpus, num_topics=num_topics, iterations=100, minimum_probability=0.01,
                                                    id2word=id2word,chunksize=2000, random_state=1,passes=50,workers=35)
        model_list.append(model)
        coherencemodel = gensim.models.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        perpexities.append(model.log_perplexity(corpus))
#         np.savetxt("lda_gridsearch15-26_rng1.csv", np.array([np.array(range(start, limit, step))[:len(perpexities)],coherence_values,perpexities]))

    return model_list, coherence_values, perpexities

In [ ]:
model_list, coherence_values, perpexities = compute_coherence_values(id2word=id2word_quad, corpus=corpus_quad, texts=GaussComps_str, start=10, limit=100, step=5)

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus_quad,
                                       id2word=id2word_quad,
                                       num_topics=1000, 
                                       random_state=0,
                                       chunksize=2000,
                                       passes=10,
                                       per_word_topics=True, 
                                       minimum_probability=0.01,
                                      workers=30)

In [ ]:
model_list2, coherence_values2, perpexities2 = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=GaussComps_str, start=15, limit=26, step=1)

In [ ]:
corpus

In [ ]:
lda23 = model_list[13]

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=GaussComps_str):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda23, corpus=corpus, texts=GaussComps_str)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

df_dominant_topic["Indices"] = df_dominant_topic.index.values

# Show
df_dominant_topic

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorted_df = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorted_df = pd.concat([sent_topics_sorted_df, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(5)], 
                                            axis=0)

# Reset Index    
# sent_topics_sorted_df.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorted_df.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorted_df

In [ ]:
sent_topics_sorted_df.index.values

In [ ]:
with open('{}/1776_light_curves_1s_bin_errorfix.pkl'.format(data_dir), 'rb') as f:
    lcs = pickle.load(f)
with open('{}/1776_light_curves_1s_bin_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    ids = pickle.load(f)

In [ ]:
plt.close()

# counts_todo = [i for i,v in class_normalized_comparison_matrix_df["omega"].sort_values(ascending=False)[:10].items() if v>=0.5]

# if len(counts_todo) > 3:
#     how_many_to_plot = 10
# else:
how_many_to_plot = 5

plt.rcParams['figure.figsize'] = (3.32*3,3.32*(2)*3)
plt.rcParams.update({'font.size': 6})

# component_counts = np.unique(shape_moments_GMM122_labels, return_counts=1)

# max_stats = np.max(shape_moments[:,-4:],axis=0)
# min_stats = np.min(shape_moments[:,-4:],axis=0)

for plot_topic in range(23):
    fig, axes = plt.subplots(nrows=how_many_to_plot, ncols=2)
#     component_segment_indices = np.where(shape_moments_GMM122_labels == plot_component)[0]
    for plot_ind in range(how_many_to_plot):
        lc_index = sent_topics_sorted_df.index.values[plot_topic*5 + plot_ind]
        data = lcs[lc_index]
#         reconstruction = segment_reconstructions[component_segment_indices[plot_ind]]
        axes[plot_ind,0].plot(data[0]-data[0][0],data[1]/1000)#, c="green", linewidth=0.5, zorder=-5)
        axes[plot_ind,0].text(0.99,0.99,"{}".format(lc_index), ha='right', va='top', transform=axes[plot_ind, 0].transAxes, size=8)
        
#         axes[plot_ind,0].plot(np.array(list(range(128)))*4, reconstruction/1000)
        axes[plot_ind,1].plot(data[0]-data[0][0],data[1]/1000)#, c="green", linewidth=0.5, zorder=-5)
#         axes[plot_ind,1].plot(np.array(list(range(128)))*4, reconstruction/1000)
    #     axes[subplot, 1].plot(np.array(list(range(128)))*4,segments_counts[class_segments[subplot]])
    #     axes[subplot, 1].plot(np.array(list(range(128)))*4, reconstruction*np.std(data)+np.mean(data))
        axes[plot_ind, 1].set_ylim([0, 12])
        axes[plot_ind, 1].set_xlim([0, 3000])
        axes[plot_ind, 1].yaxis.tick_right()
        
#         bar_heights=np.zeros(4)
#         bar_heights[0] = np.mean(data)
#         bar_heights[1] = np.std(data)
#         bar_heights[2] = stats.skew(data)
#         bar_heights[3] = stats.kurtosis(data)

#         color = ["red" if x else "blue" for x in shape_moments[component_segment_indices[plot_ind],-4:]<0]
#         bar_heights = (shape_moments[component_segment_indices[plot_ind],-4:]-min_stats)/ (max_stats-min_stats)
#         axes[plot_ind, 2].bar(x=range(4), height=bar_heights)#, color=color)
#         axes[plot_ind, 2].set_ylim([0,1])#[np.min(shape_moments[:,-4:]), np.max(shape_moments[:,-4:])])

    #     if class_name == "Unknown": 
    #         class_name = "??"
    #     else:
    #         class_name = r"$\{}$".format(class_name)
    #     axes[subplot, 1].text(x=510, y=9500, s=class_name)

        if plot_ind == 2:
            axes[plot_ind,0].set_ylabel("Rate (kcounts/s)", size=6)
        if plot_ind == 4:#plot_ind == 12 or plot_ind == 13
            axes[plot_ind,0].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
            axes[plot_ind,1].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
            axes[plot_ind,0].set_xlabel("Time (s)", size=6, x=1)

        else:
            axes[plot_ind,0].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=0, direction="in")
            axes[plot_ind,1].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=0, direction="in")

    plt.suptitle("component {}".format(plot_topic), y=1.01)
    plt.tight_layout()
    plt.show()


In [ ]:
lda23.get_document_topics(corpus)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda23, corpus, id2word)
vis

In [ ]:
limit=10; start=30; step=1;
x = range(10, 30, 1)
plt.plot(x, coherence_values, label="coherence_values")
x = range(15, 26, 1)
plt.plot(x, coherence_values2, label="coherence_values")
# plt.plot(x, perpexities, label="perpexities")
plt.xlabel("Num Topics")
plt.ylabel("score")
plt.xlim((0,50))
plt.legend()
plt.show()

In [ ]:
np.array(range(10, 30, 1))[np.argsort(coherence_values)]

In [ ]:
np.argsort(coherence_values)

In [ ]:
np.array(list(range(start, limit, step)))[np.argsort(perpexities)]

In [ ]:
np.array(list(range(start, limit, step)))

In [ ]:
corpus_quad = [] # list of lists of tuples with word id and count within observation
for observation in GaussComps_str:
    ind_counts = [(ind, count)for ind, count in enumerate(np.array(np.unique(observation, return_counts=True ))[1])]
    corpus_quad.append(ind_counts)

In [ ]:
[(ind, count)for ind, count in enumerate(np.array(np.unique(GaussComps_str[0], return_counts=True ))[1])]

In [ ]:

print(( np.unique(data_words_quadgrams[0], return_counts=True)[0]).shape)
np.unique(data_words_quadgrams[0], return_counts=True)

In [ ]:
np.unique([item for sublist in data_words_quadgrams for item in sublist])

In [ ]:
len(data_words_quadgrams)

In [ ]:
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))  # deacc=True removes punctuationsdata = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(GaussComps_str))
print(data_words[:1])

In [ ]:
texts = GaussComps_str
bigram = gensim.models.Phrases(texts)
texts = [bigram[line] for line in texts]

In [ ]:
texts[0]

In [ ]:
[trigram_mod[bigram_mod[doc]] for doc in GaussComps_str[0]]

In [ ]:
# for i in bigram_mod[ObID_GaussComps_dict_122.values()]:
#     print(i)
    
print(bigram_mod[GaussComps_mt2[0]])

In [ ]:
bigram.save_corpus("corpus", ObID_GaussComps_dict_122.values())

In [ ]:
import gensim
# from gensim.utils import simple_preprocess


In [ ]:
import gensim.corpora as corpora# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)# Create Corpus
texts = data_lemmatized# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]# View
print(corpus[:1])

In [ ]:


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



In [ ]:
from gensim.models import CoherenceModel# Compute Coherence Score
lda_model = gensim.models.LdaMulticore(corpus=corpus, #(word_id, word_frequency)
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()print('\nCoherence Score: ', coherence_lda)

In [ ]:
train_scores = []
val_scores = []
for comps, scores in results:
    train_scores.append(scores['train_score'])
    val_scores.append(scores['test_score'])

In [ ]:
np.array(train_scores).shape

In [ ]:
for iteration in range(5):
    plt.plot(np.array(range(20))+1, np.array(train_scores)[:,iteration])
plt.show()

In [ ]:
results

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

# Visualise agglomerative cluster populations

In [ ]:
clusters = sch.fcluster(linkage, 0.98298298, criterion='distance')

In [ ]:
plt.close()

# counts_todo = [i for i,v in class_normalized_comparison_matrix_df["omega"].sort_values(ascending=False)[:10].items() if v>=0.5]

# if len(counts_todo) > 3:
#     how_many_to_plot = 10
# else:


plt.rcParams['figure.figsize'] = (3.32*3,3.32*(2)*2)
plt.rcParams.update({'font.size': 6})

lcs_agglo_clusters = clusters-1 # output of fcluster is 1 indexed # np.unique(shape_moments_GMM122_labels, return_counts=1)

max_stats = np.max(shape_moments[:,-4:],axis=0)
min_stats = np.min(shape_moments[:,-4:],axis=0)

for plot_component in range(60):
    
    cluster_lc_indices = np.where(lcs_agglo_clusters == plot_component)[0]
    cluster_lc_ids = obs_component_counts_df_122.index.values[cluster_lc_indices]
    how_many_to_plot = len(cluster_lc_indices)
    fig, axes = plt.subplots(nrows=how_many_to_plot, ncols=2)
    plt.rcParams['figure.figsize'] = (3.32*2,3.32*4*(2/3)*(how_many_to_plot/10))

    for plot_ind in range(how_many_to_plot):
        data = lcs[np.where(np.array(ids) == cluster_lc_ids[plot_ind])[0][0]]
#         reconstruction = segment_reconstructions[component_segment_indices[plot_ind]]
        axes[plot_ind,0].plot(data[0]-data[0][0],data[1]/1000)#, c="green", linewidth=0.5, zorder=-5)
#         axes[plot_ind,0].plot(np.array(list(range(128)))*4, reconstruction/1000)
        axes[plot_ind,1].plot(data[0][:3000]-data[0][0],data[1][:3000]/1000)#, c="green", linewidth=0.5, zorder=-5)
#         axes[plot_ind,1].plot(np.array(list(range(128)))*4, reconstruction/1000)
    #     axes[subplot, 1].plot(np.array(list(range(128)))*4,segments_counts[class_segments[subplot]])
    #     axes[subplot, 1].plot(np.array(list(range(128)))*4, reconstruction*np.std(data)+np.mean(data))
        axes[plot_ind, 1].set_ylim([0, 12])
#         axes[plot_ind, 1].set_xlim([0, 3000])
        axes[plot_ind, 1].yaxis.tick_right()
        
#         bar_heights=np.zeros(4)
#         bar_heights[0] = np.mean(data)
#         bar_heights[1] = np.std(data)
#         bar_heights[2] = stats.skew(data)
#         bar_heights[3] = stats.kurtosis(data)

#         color = ["red" if x else "blue" for x in shape_moments[component_segment_indices[plot_ind],-4:]<0]
#         bar_heights = (shape_moments[component_segment_indices[plot_ind],-4:]-min_stats)/ (max_stats-min_stats)
#         axes[plot_ind, 2].bar(x=range(4), height=bar_heights)#, color=color)
#         axes[plot_ind, 2].set_ylim([0,1])#[np.min(shape_moments[:,-4:]), np.max(shape_moments[:,-4:])])

    #     if class_name == "Unknown": 
    #         class_name = "??"
    #     else:
    #         class_name = r"$\{}$".format(class_name)
    #     axes[subplot, 1].text(x=510, y=9500, s=class_name)

#         if plot_ind == 4:
#             axes[plot_ind,0].set_ylabel("Rate (kcounts/s)", size=6)
        if plot_ind == len(cluster_lc_indices)-1:#plot_ind == 12 or plot_ind == 13
            axes[plot_ind,0].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
            axes[plot_ind,1].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
# #             axes[plot_ind,0].set_xlabel("Time (s)", size=6, x=1)

        else:
#             axes[plot_ind,0].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=0, direction="in")
#             axes[plot_ind,1].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=0, direction="in")
            axes[plot_ind,0].set_xticklabels([])
            axes[plot_ind,1].set_xticklabels([])
            
        if plot_ind == len(cluster_lc_indices)-1:
            break

    plt.suptitle("component {}, population {}".format(plot_component, len(cluster_lc_indices)))
    plt.savefig('figures/agglo_clusters/component{}population{}.png'.format(plot_component, len(cluster_lc_indices)), dpi=100, bbox_inches = 'tight',pad_inches = 0)
    plt.tight_layout()
    plt.show()


In [ ]:
lcs[np.where(np.array(ids) == cluster_lc_ids[plot_ind])]

In [ ]:
np.where(np.array(ids) == cluster_lc_ids[plot_ind])[0]

In [ ]:
np.unique(lcs_agglo_clusters)

In [ ]:
y_limit = linkage[-1][2]
no_clusters = []
for distance in np.linspace(0,y_limit,1000):
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    no_clusters.append(len(np.unique(clusters)))

plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
plt.plot(np.linspace(0,y_limit,1000), no_clusters, c="magenta")
plt.title("No. of clusters as the function of distance for hierarchical clustering")
plt.ylabel("No. clusters")
plt.xlabel("Distance threshold")
# plt.ylim((0,50))
plt.show()

In [ ]:
linkage = sch.linkage(non_zero_row_ratio, method="complete", metric=aitchison)
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=1, leaf_font_size = 0)
plt.show()

In [ ]:
y_limit = linkage[-1][2]
no_clusters = []
for distance in np.linspace(0,y_limit,1000):
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    no_clusters.append(len(np.unique(clusters)))

plt.rcParams['figure.figsize'] = [5,5]
# plt.axhline(114, c="cyan")
plt.plot(np.linspace(0,y_limit,1000), no_clusters, c="magenta")
plt.title("No. of clusters as the function of distance for hierarchical clustering")
plt.ylabel("No. clusters")
plt.xlabel("Distance threshold")
# plt.ylim((0,50))
plt.show()

In [ ]:
no_clusters[-2]

In [ ]:
plt.scatter(obs_component_counts_df_122.iloc[:,0], obs_component_counts_df_122.iloc[:,4])
plt.show()

In [ ]:
plt.scatter(normalized_obs_component_counts_df_122.iloc[:,0], normalized_obs_component_counts_df_122.iloc[:,4])
plt.show()

In [ ]:
plt.scatter(transformed_features[:,0], transformed_features[:,1])
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


In [ ]:
import scipy.cluster.hierarchy as sch

def plot_denogram_and_no_clusters_curve(method, metric, data_set, data_set_name):
    """
    method : how are the clusters agglomerated, eg. merge the two nearest clusters w.r.t the furthest data point("complete"), 
                cluster centroid ("average"), minimum variance ("ward")
    metric : euclidean, cosine (angular metric ignores the vector magnitude, so can be helpful for our counts)
    data_set : one of the data frames with shape [1738, 500]
    data_set_name : row_standardised, column_standardised, row_normalised, raw_counts
    """
    
    if method == "ward": # must be euclidean
        metric = "euclidean"
    
    print("Method : {}, Metric : {}, Data : {}".format(method, metric, data_set_name))
    plt.rcParams['figure.figsize'] = [9,3]
    linkage = sch.linkage(data_set, method=method, metric=metric)
    dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=2, leaf_font_size = 0)
    y_limit = linkage[-1][2] # distance value of the last merger
    ax=plt.gca()
    ax.set_ylabel("Euclidean distance")
    ax.set_xlabel("Observations of the source (1738)")
    ax.set_xticklabels(())
    plt.title("Method : {}, Metric : {}, Data : {}".format(method, metric, data_set_name))
    plt.show()

    # find no. clusters as a function of distance
    no_clusters = []
    for distance in np.linspace(0,y_limit,1000):
        clusters = sch.fcluster(linkage, distance, criterion='distance')
        no_clusters.append(len(np.unique(clusters)))

    plt.rcParams['figure.figsize'] = [5,5]
    plt.axhline(114, c="cyan")
    plt.plot(np.linspace(0,y_limit,1000), no_clusters, c="magenta")
    plt.title("No. of clusters as the function of distance for hierarchical clustering")
    plt.ylabel("No. clusters")
    plt.xlabel("Distance threshold")
    plt.show()

In [ ]:
plot_denogram_and_no_clusters_curve("complete", "cosine", obs_component_counts_df_122, "raw_counts")

In [ ]:
method = "complete"
metric = "cosine"
data_set = obs_component_counts_df_122
data_set_name = "raw_counts"

print("Method : {}, Metric : {}, Data : {}".format(method, metric, data_set_name))
plt.rcParams['figure.figsize'] = [9,3]
linkage = sch.linkage(data_set, method=method, metric=metric)
dendrogram = sch.dendrogram(linkage, truncate_mode=None, p=2, leaf_font_size = 0)
y_limit = linkage[-1][2] # distance value of the last merger
ax=plt.gca()
ax.set_ylabel("Euclidean distance")
ax.set_xlabel("Observations of the source (1738)")
ax.set_xticklabels(())
plt.title("Method : {}, Metric : {}, Data : {}".format(method, metric, data_set_name))
plt.show()

# find no. clusters as a function of distance
no_clusters = []
for distance in np.linspace(0,y_limit,1000):
    clusters = sch.fcluster(linkage, distance, criterion='distance')
    no_clusters.append(len(np.unique(clusters)))

plt.rcParams['figure.figsize'] = [5,5]
plt.axhline(73, c="cyan")
plt.plot(np.linspace(0,y_limit,1000), no_clusters, c="magenta")
plt.title("No. of clusters as the function of distance for hierarchical clustering")
plt.ylabel("No. clusters")
plt.xlabel("Distance threshold")
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]
plt.axhline(114, c="cyan")
plt.plot(np.linspace(0,y_limit,1000), no_clusters, c="magenta")
plt.xlim([0.65,1])
plt.ylim((0,120))
plt.title("No. of clusters as the function of distance for hierarchical clustering")
plt.ylabel("No. clusters")
plt.xlabel("Distance threshold")
plt.show()

In [ ]:
for i in range(100):
    print(np.linspace(0,y_limit,1000)[-i], no_clusters[-i])

In [ ]:
from sklearn.cluster import AgglomerativeClustering

clustering = AgglomerativeClustering(n_clusters=55, linkage=method, affinity=metric).fit(data_set )
clustering

ob_clusters_55_labels  = clustering.labels_

In [ ]:
# load observation classifications from Huppenkothen 2017
clean_belloni = open('{}/1915Belloniclass_updated.dat'.format(data_dir))
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
    if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state
        
# make a dict of Tomaso's classifications  (Daniela's set) against observation ids,
# i.e. ob_state == {'20187-02-01-00': 'alpha', '20187-02-01-01': 'alpha', '20402-01-22-00': 'alpha', ...}
# lines = clean_belloni.readlines()
# states = lines[0].split()
# belloni_clean = {}
# for h,l in zip(states, lines[1:]):
#     belloni_clean[h] = l.split()
#     #state: obsID1, obsID2...
# ob_state = {}
# for state, obs in belloni_clean.items():
#     if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
#     for ob in obs:
#         ob_state[ob] = state
        
        
# load IDs of segmented light curves: observationsID_segmentIndex
with open('{}/468202_len128_s2_4cad_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    seg_ids = pickle.load(f)

        
seg_ObIDs = [seg.split("_")[0] for seg in seg_ids] # get rid of the within-observation segment indices and create a degenerate list of observation IDs

classes = np.array(["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega"])
scales = []
segment_class = []
for ob in seg_ObIDs:
    if ob in ob_state:
        segment_class.append(ob_state[ob])
    else:
        segment_class.append("Unknown")

In [ ]:
with open('{}/1776_light_curves_1s_bin_errorfix.pkl'.format(data_dir), 'rb') as f:
    lcs = pickle.load(f)
with open('{}/1776_light_curves_1s_bin_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    ids = pickle.load(f)

In [ ]:
#nu 38 60
#eta 78, 85
cluster_no = 0
nrows = 10
cluster_labels = ob_clusters_53_labels
data_set = pseudo_CLR_transformed
plt.close()
plt.tight_layout()
plt.rcParams['figure.figsize'] = (6.97, 8.4)
plt.rcParams.update({'font.size': 6})

for cluster_no in np.unique(cluster_labels):
    fig, axes = plt.subplots(nrows=nrows, ncols=2)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.6)
#     plt.suptitle("cluster {}".format(cluster_no), y=0.9)
    lcs_of_interest = data_set.index.values[np.where(cluster_labels == cluster_no)[0]]#
    # lcs_of_interest = inv_ob_state["beta"]#
    for n_plot, lc_oi in enumerate(lcs_of_interest):
        index_oi = np.where(np.array(ids) == lc_oi)[0][0]
        axes[n_plot%nrows,0].plot(lcs[index_oi][0]-lcs[index_oi][0][0], lcs[index_oi][1], c= "black", linewidth=0.5)
        axes[n_plot%nrows, 1].plot(lcs[index_oi][0]-lcs[index_oi][0][0], lcs[index_oi][1]/1000, c= "black", linewidth=0.5)
        axes[n_plot%nrows,0].set_xlim([0,2000])
        axes[n_plot%nrows,0].set_ylim([500,13000])
        axes[n_plot%nrows,0].set_yticks([0,2000,4000,6000 , 8000])
        axes[n_plot%nrows,0].set_yticklabels([0, "", "", "", 8])
        axes[n_plot%nrows, 1].yaxis.tick_right()
        axes[n_plot%nrows, 0].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
        axes[n_plot%nrows, 1].tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

        if n_plot == 4:
            axes[n_plot,0].set_ylabel("X-ray rate (kcounts/s)")
        elif n_plot == 9:
            axes[n_plot,0].set_xlabel("Time (s)", x=1.1)
        if (n_plot%(nrows-1) == 0) and n_plot>0:
            plt.savefig('figures/agglo_clusters/set53_c{}.png'.format(cluster_no), dpi=300, bbox_inches = 'tight',pad_inches = 0)
            plt.show()
            break
            fig, axes = plt.subplots(nrows=nrows, ncols=1)
            plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
    #         plt.suptitle("cluster {}".format(cluster_no), y=0.9)
        elif (n_plot == len(lcs_of_interest)-1):
            plt.show()


In [ ]:
plot_cluster = 0
np.where(ob_clusters_55_labels==plot_cluster)[0]

In [ ]:
# clustering = AgglomerativeClustering(n_clusters=73, linkage=method, affinity=metric).fit(data_set )
# clustering

# ob_clusters_55_labels  = clustering.labels_
ob_clusters_53_labels = sch.fcluster(linkage, kn.knee, criterion='distance')

In [ ]:
# load observation classifications from Huppenkothen 2017
clean_belloni = open('{}/1915Belloniclass_updated.dat'.format(data_dir))
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
    if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state
        
# load IDs of segmented light curves: observationsID_segmentIndex
with open('{}/468202_len128_s2_4cad_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    seg_ids = pickle.load(f)

        
seg_ObIDs = [seg.split("_")[0] for seg in seg_ids] # get rid of the within-observation segment indices and create a degenerate list of observation IDs

classes = np.array(["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega"])
scales = []
segment_class = []
for ob in seg_ObIDs:
    if ob in ob_state:
        segment_class.append(ob_state[ob])
    else:
        segment_class.append("Unknown")

In [ ]:
cluster_label_df = pd.DataFrame(ob_clusters_53_labels, index=pseudo_CLR_transformed.index.values)

In [ ]:
np.unique(cluster_label_df)

In [ ]:
ob_clusters

In [ ]:
new_classification = cluster_label_df # they're in the order of pseudo_CLR_transformed.index.values

ob_classes = []
ob_clusters = []
for ob in np.unique(seg_ObIDs):
    ob_clusters.append(cluster_label_df.loc[ob][0])
    if ob in ob_state:
        ob_classes.append(ob_state[ob])
    else:
        ob_classes.append("Unknown")

Belloni_classes = np.array(["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega", "Unknown"])

comparison_matrix = np.zeros((len(np.unique(new_classification)), len(Belloni_classes)), dtype=int)

comparison_matrix_df = pd.DataFrame(comparison_matrix, columns=Belloni_classes, index=np.unique(new_classification))

for n_Bc, Belloni_class in enumerate(Belloni_classes):
    Belloni_class_indices = np.where(np.array(ob_classes) == Belloni_class)[0]
    count_clusters_for_class = np.unique(np.take(ob_clusters, Belloni_class_indices), return_counts=True)
    for cluster_ind, cluster in enumerate(count_clusters_for_class[0]):
        comparison_matrix_df[Belloni_class][cluster] = count_clusters_for_class[1][cluster_ind]

In [ ]:
class_normalized_comparison_matrix_df=(comparison_matrix_df-comparison_matrix_df.min())/(comparison_matrix_df.max()-comparison_matrix_df.min())
known_comparison_matrix_df = comparison_matrix_df.drop(columns=['Unknown']).T
component_normalized_comparison_matrix_df = (known_comparison_matrix_df-known_comparison_matrix_df.min())/(known_comparison_matrix_df.max()-known_comparison_matrix_df.min())

In [ ]:
np.array([(str(x), "") for x in list(range(1,54,2))]).flatten()

In [ ]:
plt.rcParams['figure.figsize'] = (6.97, 3.3)
plt.rcParams.update({'font.size': 6})


import seaborn as sns

ax = sns.heatmap(class_normalized_comparison_matrix_df.T, xticklabels=True, yticklabels=True, cmap='coolwarm')#, linewidth=0.5)
# ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 7)
# plt.title("Agglomerative clusters in terms of classified data (class-wise normalisation)")
ax.set_xlabel("Agglomerative cluster index")
ax.set_ylabel("Class of the 14 class system")
ax.set_xticklabels(np.array([(str(x), "") for x in list(range(1,54,2))]).flatten()[:-1])

plt.savefig('figures/Agglo53vsBelloni.png', dpi=300, bbox_inches = 'tight',pad_inches = 0, transparent=True)


plt.show()

In [ ]:
ids_ar = np.array(segment_class)

# class_names = list(inv_ob_state.keys())


alpha = np.where(ids_ar == "alpha")[0][1]#1
beta= np.where(ids_ar == "beta")[0][7]#7
gamma=np.where(ids_ar == "gamma")[0][4]
delta=np.where(ids_ar == "delta")[0][5]
theta=np.where(ids_ar == "theta")[0][4]#4
kappa=np.where(ids_ar == "kappa")[0][3]
lambda1=np.where(ids_ar == "lambda")[0][2]
mu=np.where(ids_ar == "mu")[0][1]
nu=np.where(ids_ar == "nu")[0][1]
rho=np.where(ids_ar == "rho")[0][1]
phi=np.where(ids_ar == "phi")[0][1]
chi=np.where(ids_ar == "chi")[0][1]
eta=np.where(ids_ar == "eta")[0][4]
omega=np.where(ids_ar == "omega")[0][3]

In [ ]:
desc_stats[delta,2]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
axes = axes.flatten()


plt.rcParams['figure.figsize'] = (3.32, 3.32*(1/1))
plt.rcParams.update({'font.size': 6})

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

stat_names = np.array(["mean", "st.d.", "skew", "kurtosis"])

x_tick_space = [2500, 1000, 1, 2]
x_limits = [[0,8000],[0,2500], [-2,3], [-2,8]]

for plot_ind in range(4):
#     light_c = selected_lcs[plot_ind]
    axes[plot_ind].hist(desc_stats[:,plot_ind], bins=50, range = x_limits[plot_ind],zorder=-4)
    y_vals = axes[plot_ind].get_yticks()
    axes[plot_ind].set_yticklabels(['{:3.0f}'.format(x / 10000) for x in y_vals])
    axes[plot_ind].xaxis.set_major_locator(MultipleLocator(x_tick_space[plot_ind]))
    axes[plot_ind].xaxis.set_major_formatter(FormatStrFormatter('%d'))
    axes[plot_ind].xaxis.set_minor_locator(AutoMinorLocator())
    axes[plot_ind].text(0.99,0.99,"{}".format(stat_names[plot_ind]), ha='right', va='top', transform=axes[plot_ind].transAxes, size=6)
    axes[plot_ind].axvline(desc_stats[chi,plot_ind], c="cyan", zorder=-3, alpha=0.5)
    axes[plot_ind].axvline(desc_stats[phi,plot_ind], c="magenta", zorder=-3, alpha=0.5)
    axes[plot_ind].axvline(desc_stats[gamma,plot_ind], c="black", zorder=-3, alpha=0.5)
    if plot_ind == 2:
        axes[plot_ind].set_ylabel("No. segments ($10^4$)", y=1, size=6)
        axes[plot_ind].set_xlabel("Value of the statistic", x=1, size=6)

axes.reshape((2,2))
plt.savefig('figures/stat_distributions.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

plt.show()

In [ ]:
plt.hist(desc_stats[:,3], bins=20)#, bins=50, range=[-2,48])
# plt.show()

In [ ]:
with open('{}/1776_light_curves_1s_bin_errorfix.pkl'.format(data_dir), 'rb') as f:
    lcs = pickle.load(f)
with open('{}/1776_light_curves_1s_bin_ids_errorfix.pkl'.format(data_dir), 'rb') as f:
    ids = pickle.load(f)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()fig, axes = plt.subplots(nrows=5, ncols=2)
axes = axes.flatten()


plt.rcParams['figure.figsize'] = (20,10)
plt.rcParams.update({'font.size': 16})

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

stat_names = np.array(["mean", "st.d.", "skew", "kurtosis"])

x_tick_space = [2500, 1000, 1, 2]
x_limits = [[0,8000],[0,2500], [-2,3], [-2,8]]

for plot_ind in range(10):
#     light_c = selected_lcs[plot_ind]
    axes[plot_ind].plot(lcs[plot_ind][0]-lcs[plot_ind][0][0], lcs[plot_ind][1]/1000, c="black")
    axes[plot_ind].set_ylim([1.2,10])
    axes[plot_ind].set_xlim([0,4000])
    axes[plot_ind].set_yticks([0,4,8])
    axes[plot_ind].set_yticklabels([0,4,8])
#     .errorbar(lcs[plot_ind][0]-lcs[plot_ind][0][0], lcs[plot_ind][1], yerr=lcs[plot_ind][2], ecolor="black")
#     .hist(desc_stats[:,plot_ind], bins=50, range = x_limits[plot_ind],zorder=-4)
#     y_vals = axes[plot_ind].get_yticks()
#     axes[plot_ind].set_yticklabels(['{:3.0f}'.format(x / 10000) for x in y_vals])
#     axes[plot_ind].xaxis.set_major_locator(MultipleLocator(x_tick_space[plot_ind]))
#     axes[plot_ind].xaxis.set_major_formatter(FormatStrFormatter('%d'))
#     axes[plot_ind].xaxis.set_minor_locator(AutoMinorLocator())
#     axes[plot_ind].text(0.99,0.99,"{}".format(stat_names[plot_ind]), ha='right', va='top', transform=axes[plot_ind].transAxes, size=6)
#     axes[plot_ind].axvline(desc_stats[chi,plot_ind], c="cyan", zorder=-3, alpha=0.5)
#     axes[plot_ind].axvline(desc_stats[phi,plot_ind], c="magenta", zorder=-3, alpha=0.5)
#     axes[plot_ind].axvline(desc_stats[gamma,plot_ind], c="black", zorder=-3, alpha=0.5)
#     if plot_ind == 2:
#         axes[plot_ind].set_ylabel("No. segments ($10^4$)", y=1, size=6)
#         axes[plot_ind].set_xlabel("Value of the statistic", x=1, size=6)
    if plot_ind == 8:
        axes[plot_ind].set_ylabel("X-ray rate (kcounts/s)", y=3.5, fontsize=20)
        axes[plot_ind].set_xlabel("Time (s)", x=1, fontsize=20)

axes.reshape((5,2))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
plt.savefig('figures/example_light_curves.png', dpi=300, bbox_inches = 'tight',pad_inches = 0, transparent=True)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

print(__doc__)

np.random.seed(42)
plt.figure(figsize=(8, 8))


# Generate train data
X_inliers = 0.3 * np.random.randn(100, 2)
X_inliers = np.r_[X_inliers + 2, X_inliers - 2]

# Generate some outliers
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))
X = np.r_[X_inliers, X_outliers]

n_outliers = len(X_outliers)
ground_truth = np.ones(len(X), dtype=int)
ground_truth[-n_outliers:] = -1

# fit the model for outlier detection (default)
clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
# use fit_predict to compute the predicted labels of the training samples
# (when LOF is used for outlier detection, the estimator has no predict,
# decision_function and score_samples methods).
y_pred = clf.fit_predict(X)
n_errors = (y_pred != ground_truth).sum()
X_scores = clf.negative_outlier_factor_

# plt.title("Local Outlier Factor (LOF)")
plt.scatter(X[:, 0], X[:, 1], color='k', s=10., label='Data points')
# plot circles with radius proportional to the outlier scores
radius = (X_scores.max() - X_scores) / (X_scores.max() - X_scores.min())
# plt.scatter(X[:, 0], X[:, 1], s=1000 * radius, edgecolors='r',
#             facecolors='none', label='Outlier scores')

gmm = mixture.GaussianMixture(n_components=2,
                              covariance_type="full")
gmm.fit(X)


splot = plt.subplot(1, 1, 1)
Y_ = gmm.predict(X)
for i, (mean, cov, color) in enumerate(zip(gmm.means_, gmm.covariances_,
                                           color_iter)):
    v, w = linalg.eigh(cov)
    if not np.any(Y_ == i):
        continue
    plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], 2.8, color=color)

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180. * angle / np.pi  # convert to degrees
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)



# plt.axis('tight')
# plt.xlim((-5, 5))
# plt.ylim((-5, 5))
# plt.xlabel("prediction errors: %d" % (n_errors))
# legend = plt.legend(loc='upper left')
# legend.legendHandles[0]._sizes = [10]
# legend.legendHandles[1]._sizes = [20]
plt.savefig('figures/example_clusters.png', dpi=300, bbox_inches = 'tight',pad_inches = 0, transparent=True)

plt.show()

In [ ]:
import numpy as np
import itertools

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture

print(__doc__)

# Number of samples per component
n_samples = 500

# Generate random sample, two components
np.random.seed(0)
# C = np.array([[0., -0.1], [1.7, .4]])
# X = np.r_[np.dot(np.random.randn(n_samples, 2), C),
#           .7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]

lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,
                                      covariance_type=cv_type)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                              'darkorange'])
clf = best_gmm
bars = []

# Plot the BIC scores
plt.figure(figsize=(8, 6))
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

# Plot the winner
splot = plt.subplot(2, 1, 2)
Y_ = clf.predict(X)
for i, (mean, cov, color) in enumerate(zip(clf.means_, clf.covariances_,
                                           color_iter)):
    v, w = linalg.eigh(cov)
    if not np.any(Y_ == i):
        continue
    plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180. * angle / np.pi  # convert to degrees
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)

plt.xticks(())
plt.yticks(())
plt.title('Selected GMM: full model, 2 components')
plt.subplots_adjust(hspace=.35, bottom=.02)
plt.show()

In [ ]:
from matplotlib import pyplot
from math import cos, sin, atan


class Neuron():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self, neuron_radius):
        circle = pyplot.Circle((self.x, self.y), radius=neuron_radius, fill=False)
        pyplot.gca().add_patch(circle)


class Layer():
    def __init__(self, network, number_of_neurons, number_of_neurons_in_widest_layer):
        self.vertical_distance_between_layers = 6
        self.horizontal_distance_between_neurons = 2
        self.neuron_radius = 0.5
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.previous_layer = self.__get_previous_layer(network)
        self.y = self.__calculate_layer_y_position()
        self.neurons = self.__intialise_neurons(number_of_neurons)

    def __intialise_neurons(self, number_of_neurons):
        neurons = []
        x = self.__calculate_left_margin_so_layer_is_centered(number_of_neurons)
        for iteration in range(number_of_neurons):
            neuron = Neuron(x, self.y)
            neurons.append(neuron)
            x += self.horizontal_distance_between_neurons
        return neurons

    def __calculate_left_margin_so_layer_is_centered(self, number_of_neurons):
        return self.horizontal_distance_between_neurons * (self.number_of_neurons_in_widest_layer - number_of_neurons) / 2

    def __calculate_layer_y_position(self):
        if self.previous_layer:
            return self.previous_layer.y + self.vertical_distance_between_layers
        else:
            return 0

    def __get_previous_layer(self, network):
        if len(network.layers) > 0:
            return network.layers[-1]
        else:
            return None

    def __line_between_two_neurons(self, neuron1, neuron2):
        angle = atan((neuron2.x - neuron1.x) / float(neuron2.y - neuron1.y))
        x_adjustment = self.neuron_radius * sin(angle)
        y_adjustment = self.neuron_radius * cos(angle)
        line = pyplot.Line2D((neuron1.x - x_adjustment, neuron2.x + x_adjustment), (neuron1.y - y_adjustment, neuron2.y + y_adjustment))
        pyplot.gca().add_line(line)

    def draw(self, layerType=0):
        for neuron in self.neurons:
            neuron.draw( self.neuron_radius )
            if self.previous_layer:
                for previous_layer_neuron in self.previous_layer.neurons:
                    self.__line_between_two_neurons(neuron, previous_layer_neuron)
        # write Text
#         x_text = self.number_of_neurons_in_widest_layer * self.horizontal_distance_between_neurons
#         if layerType == 0:
#             pyplot.text(x_text, self.y, 'Input Layer', fontsize = 12)
#         elif layerType == -1:
#             pyplot.text(x_text, self.y, 'Output Layer', fontsize = 12)
#         else:
#             pyplot.text(x_text, self.y, 'Hidden Layer '+str(layerType), fontsize = 12)

class NeuralNetwork():
    def __init__(self, number_of_neurons_in_widest_layer):
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.layers = []
        self.layertype = 0

    def add_layer(self, number_of_neurons ):
        layer = Layer(self, number_of_neurons, self.number_of_neurons_in_widest_layer)
        self.layers.append(layer)

    def draw(self):
        pyplot.figure()
        for i in range( len(self.layers) ):
            layer = self.layers[i]
            if i == len(self.layers)-1:
                i = -1
            layer.draw( i )
        pyplot.axis('scaled')
        pyplot.axis('off')
#         pyplot.title( 'Neural Network architecture', fontsize=15 )
        pyplot.savefig('figures/neural_net.png', dpi=300, bbox_inches = 'tight',pad_inches = 0, transparent=True)
        pyplot.show()

class DrawNN():
    def __init__( self, neural_network ):
        self.neural_network = neural_network

    def draw( self ):
        widest_layer = max( self.neural_network )
        network = NeuralNetwork( widest_layer )
        for l in self.neural_network:
            network.add_layer(l)
        network.draw()

In [ ]:
network = DrawNN( [12,8,4] )
network.draw()


In [ ]:
# with open('../../../data_GRS1915/1776_light_curves_1s_bin_errorfix.pkl', 'rb') as f:
#     lcs = pickle.load(f)
# with open('../../../data_GRS1915/1776_light_curves_1s_bin_ids_errorfix.pkl', 'rb') as f:
#     ids = pickle.load(f)
    
    
clean_belloni = open('{}/1915Belloniclass_updated.dat'.format(data_dir))
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
    if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state

        

inv_ob_state = {}
for k, v in ob_state.items():
    inv_ob_state[v] = inv_ob_state.get(v, [])
    inv_ob_state[v].append(k)


import matplotlib.pylab as pylab
import matplotlib.ticker as ticker
#https://stackoverflow.com/questions/8389636/creating-over-20-unique-legend-colors-using-matplotlib


pylab.rcParams['figure.figsize'] = (29.7, 21.0) # A4 size 210mm x 297mm
import matplotlib
colors = matplotlib.colors.CSS4_COLORS.keys()
colors = np.array(list(colors))


NUM_COLORS = 14
cm = plt.get_cmap("jet")#('gist_rainbow')
colors = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]

ids_ar = np.array(ids)

class_names = list(inv_ob_state.keys())


alpha = lcs[np.where(ids_ar == inv_ob_state["alpha"][0])[0][0]]
beta= lcs[np.where(ids_ar == inv_ob_state["beta"][5])[0][0]] #3
gamma=lcs[np.where(ids_ar == inv_ob_state["gamma"][0])[0][0]]
delta=lcs[np.where(ids_ar == inv_ob_state["delta"][9])[0][0]]
theta=lcs[np.where(ids_ar == inv_ob_state["theta"][13])[0][0]]#11
kappa=lcs[np.where(ids_ar == inv_ob_state["kappa"][6])[0][0]]#6
lambda1=lcs[np.where(ids_ar == inv_ob_state["lambda"][3])[0][0]] #3
mu=lcs[np.where(ids_ar == inv_ob_state["mu"][6])[0][0]]#6
nu=lcs[np.where(ids_ar == inv_ob_state["nu"][0])[0][0]]
rho=lcs[np.where(ids_ar == inv_ob_state["rho"][9])[0][0]]#9
phi=lcs[np.where(ids_ar == inv_ob_state["phi"][3])[0][0]]# 3,6
chi=lcs[np.where(ids_ar == inv_ob_state["chi"][27])[0][0]]# 1,17,27
eta=lcs[np.where(ids_ar == inv_ob_state["eta"][2])[0][0]]# 1
# omega=lcs[np.where(ids_ar == inv_ob_state["kappa"][-3])[0][0]]
omega=lcs[np.where(ids_ar == inv_ob_state["omega"][1])[0][0]]


selected_lcs = [alpha,beta,gamma,delta,theta,kappa,lambda1,mu,nu,rho,phi,chi,eta,omega]


fig, axes = plt.subplots(nrows=7, ncols=2)
axes = axes.flatten()

plt.subplots_adjust(hspace=0.05)
plt.subplots_adjust(wspace=0.01)

good_classes = ["delta", "mu", "rho", "phi"]
intervals = {}

colors = ["black"]*14

for plot_ind in range(14):
    light_c = np.copy(selected_lcs[plot_ind])
    class_name = class_names[plot_ind]
    offset = light_c[0][0]
    axes[plot_ind].set_ylim([-0.1, 1.1])
    
    if class_name == "alpha":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1
        start=0
        end =breaks[0]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][:breaks[0]]-offset, light_c[1][:breaks[0]], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].set_xlim([0, 3500])
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)

    elif class_name == "beta":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start=breaks[0]
        end =breaks[1]
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "gamma":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start=breaks[0]
        end =breaks[1]
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "theta":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start=breaks[1]
        end =breaks[2]
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "kappa":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start=breaks[-1]
        end =-1
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "lambda":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start=breaks[-1]
        end =-1
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "nu":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start= breaks[0]
        end =breaks[1]
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "chi":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start= breaks[0]
        end =-1
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "eta":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start= breaks[2]
        end =breaks[3]
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "omega":
        breaks = np.where((light_c[0][1:]-light_c[0][:-1]) != 1.)[0]+1 # [ 279 3584 6652]
        start= breaks[0]
        end =-1
        offset = light_c[0][start]
        light_c[1] /= np.max(light_c[1][start:end])*1.1
        axes[plot_ind].plot(light_c[0][start:end]-offset, light_c[1][start:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].plot(light_c[0][start:end-48]-offset+1607, light_c[1][start+48:end], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
    elif class_name == "rho":
        light_c[1] /= np.max(light_c[1])*1.1
        axes[plot_ind].plot(light_c[0]-offset, light_c[1], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
        
        
    elif class_name in good_classes:
        
        light_c[1] /= np.max(light_c[1])*1.1
        axes[plot_ind].plot(light_c[0]-offset, light_c[1], c=colors[plot_ind], linewidth=1, zorder=-5)
        axes[plot_ind].text(1,1,r"$\{}$".format(class_name), ha='right', va='top', transform=axes[plot_ind].transAxes, size=30)
        axes[plot_ind].set_xlim([0, 3500])
    else:
        axes[plot_ind].plot(light_c[0]-offset, light_c[1])
        axes[plot_ind].plot(light_c[0][:3500]-offset, light_c[1][:3500])
    
    axes[plot_ind].set_xlim([0, 2500])
#     axes[plot_ind].tick_params(axis="x", which="major", length=5, width=1, labelsize=20, direction="in")
    
    if plot_ind%2 == 0:
        axes[plot_ind].tick_params(axis="y", which="major", length=5, width=1, labelsize=20, direction="in")
    else:
        axes[plot_ind].tick_params(axis="y", which="major", length=5, width=1, labelsize=0, direction="in")
        plt.setp(axes[plot_ind].get_yticklabels(), visible=False)
    if plot_ind == 6:
        axes[plot_ind].set_ylabel("Normalised count rate", size=30)
    if plot_ind == 12:
        axes[plot_ind].tick_params(axis="x", which="major", length=5, width=1, labelsize=20, direction="in")
        axes[plot_ind].set_xlabel("Time (s)", x=1, size=30)

    else:
        axes[plot_ind].tick_params(axis="x", which="major", length=5, width=1, labelsize=0, direction="in")
        plt.setp(axes[plot_ind].get_xticklabels(), visible=False)
    
    axes[plot_ind].set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    axes[plot_ind].set_yticklabels(["0","","","", "1"])
    axes[plot_ind].set_xticks([0, 500, 1000,1500, 2000])
    axes[plot_ind].set_xticklabels([0,"",1000,"", 2000])

    
axes.reshape((7,2))


# axes[0][0].tick_params(axis="x", which="major", length=5, width=1, labelsize=5, direction="in")
# axes[0][1].tick_params(axis="x", which="major", length=5, width=1, labelsize=5, direction="in")
# axes[1][0].tick_params(axis="x", which="major", length=5, width=1, labelsize=25, direction="in")
# axes[1][1].tick_params(axis="x", which="major", length=5, width=1, labelsize=25, direction="in")

# axes[0][0].tick_params(axis="y", which="major", length=5, width=1, labelsize=25, direction="in")
# axes[1][0].tick_params(axis="y", which="major", length=5, width=1, labelsize=25, direction="in")
# axes[0][1].tick_params(axis="y", which="major", length=5, width=1, labelsize=5, direction="in")
# axes[1][1].tick_params(axis="y", which="major", length=5, width=1, labelsize=5, direction="in")

# plt.setp(axes[0][1].get_yticklabels(), visible=False)
# plt.setp(axes[1][1].get_yticklabels(), visible=False)
# plt.setp(axes[0][1].get_xticklabels(), visible=False)
# plt.setp(axes[0][0].get_xticklabels(), visible=False)

# axes[1][0].set_xticks([500,1000,1500,2000])
# axes[1][1].set_xticks([500,1000,1500,2000])


# axes[0][0].yaxis.set_label_coords(-0.2, 0)
# axes[1][0].xaxis.set_label_coords(1, -0.1)

# plt.suptitle("14 classes of activity of x-ray black hole binary GRS1915+105", fontsize=40, y=0.92)


plt.savefig('figures/all_classes_of_GRS1915.png', dpi=300, bbox_inches = 'tight',pad_inches = 0, transparent=True)

plt.show()

# Classification based on Gaussian component proportions

In [ ]:
inv_ob_state

In [ ]:
compar_results = np.zeros((2,1))

# inverse the ob_state dictionary, so that inv_ob_state contains {"state name" : [list of observation IDs], ...}
inv_ob_state = {} 
for k, v in ob_state.items():
    inv_ob_state[v] = inv_ob_state.get(v, [])
    inv_ob_state[v].append(k)

test_sets = []
train_sets = []
# split the observations into test and training sets
for test_iter in range(1000):
    test_set = []
    train_set = []
    
    # split is stratified, so done separately for each class
    for class_name in ["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega"]:

        class_obs_all = inv_ob_state[class_name] # all labeled observation IDs of this class
        # exclude observations which did not produce any light curve segments
        class_obs = []
        for ob in class_obs_all:
            if ob in seg_ObIDs: # seg_ObIDs contains observation ID for each of the 468202 light curve segments
                class_obs.append(ob)
        
        # pick 1/3 of observations for the test set (previously replace=True was set!!!!)
        test_obs = np.random.choice(class_obs, size=int(np.ceil(len(class_obs)/3)), replace=False) 

        if len(test_obs) == 0:
            print(class_name)
        
        # use the remaining observations as training set
        train_obs = []
        for ob in class_obs:
            if ob not in test_obs:
                train_obs.append(ob)
        test_set.append(test_obs)
        train_set.append(train_obs)

    test_set=np.hstack(test_set)
    train_set=np.hstack(train_set)
    test_sets.append(test_set)
    train_sets.append(train_set)


In [ ]:
len(seg_ObIDs)

In [ ]:
test_sets

In [ ]:
train_set.shape

In [ ]:
pseudo_CLR_transformed

In [ ]:
obs_component_counts_df_122

In [ ]:
print(obs_component_counts_df_122["Class"].values[:100])

In [ ]:
pseudo_CLR_transformed["Class"] = "Unknown"
for k,v in ob_state.items():
    if str(k) in obs_component_counts_df_122.index.values:
        pseudo_CLR_transformed.loc[str(k), "Class"] = v

In [ ]:
obs_component_counts_df_122.iloc[:,:122].div(np.sum(obs_component_counts_df_122.iloc[:,:122], axis=1), axis="rows")
obs_component_counts_df_122["Class"] = "Unknown"

In [ ]:
for k,v in ob_state.items():
    if str(k) in obs_component_counts_df_122.index.values:
        obs_component_counts_df_122.loc[str(k), "Class"] = v

In [ ]:
obs_component_counts_df_122

In [ ]:
pseudo_CLR_transformed

In [ ]:
len(list(ob_state.items()))

In [ ]:
148+72

In [ ]:
len(test_set)

In [ ]:
obs_component_counts_df_122.loc[test_set].Class.values

In [ ]:
inv_ob_state

In [ ]:
for t in test_set:
    print(ob_state[t])

In [ ]:
obs_component_counts_df_122.iloc[:,:122] = obs_component_counts_df_122.iloc[:,:122].div(np.sum(obs_component_counts_df_122.iloc[:,:122], axis=1), axis="rows")

In [ ]:
from sklearn.svm import SVC
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
#(y_true, y_pred, *, beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')

# import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (6.97, 6.97)
plt.rcParams.update({'font.size': 6})

experiment_list = [

    [obs_component_counts_df_122, "Classification with proportional make up of observations in terms of 114 Gaussian mixture components"],

]

reports = []
for test_set, train_set in zip(test_sets, train_sets):

    for dataset, title in experiment_list:
        SVC_clf = RandomForestClassifier(random_state=0, class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])
        if title[-8:] == "balanced":
            SVC_clf = SVC(gamma="auto", class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])

        preds = SVC_clf.predict(dataset.loc[test_set].iloc[:,:122])
#         reports.append(classification_report(dataset["Class"].loc[test_set], preds))
        reports.append(( precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="weighted"),precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="macro"), accuracy_score(dataset["Class"].loc[test_set], preds)))
#             precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="macro"), precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="micro")))
#         print("{}".format(title))
#         print(classification_report(dataset["Class"].loc[test_set], preds))



#         disp = plot_confusion_matrix(SVC_clf, dataset.loc[test_set].iloc[:,:122], dataset["Class"].loc[test_set],
#                                  cmap=plt.cm.Blues,
#                                  normalize=None)
    #     disp.ax_.set_title("{}".format(title))
#         disp.ax_.set_xlabel("Predicted class")
#         disp.ax_.set_ylabel("True class")
    #     plt.savefig('figures/classification_matrix.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

    #     plt.show()
    
    

In [ ]:
print(np.mean(f1s_weighted), np.median(f1s_weighted))

In [ ]:
print(np.mean(accuracies), np.median(accuracies))

In [ ]:
accuracies = []
f1s_weighted = []
f1s_average = []
for report in reports:# weighted(precision, recall, f1), accuracy
    accuracies.append(report[2])
    f1s_weighted.append(report[0][2])
    f1s_average.append(report[1][2])

In [ ]:
plt.rcParams['figure.figsize'] = (3.32, 3.32)
plt.rcParams.update({'font.size': 6})

plt.hist(f1s_average, bins=15, label="average F1")
plt.hist(f1s_weighted, bins=15, label="weighted F1", alpha=0.7)
plt.hist(accuracies, bins=15, label="accuracy", alpha=0.7)
plt.legend()
# plt.xlim((0.65,0.95))
axes = plt.gca()
axes.tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
axes.tick_params(axis="y", which="major", length=2, width=0.75, labelsize=6, direction="in")
plt.xlabel("Test score")
plt.ylabel("No. classification experiments")


# plt.savefig('figures/classification_histogram.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
[i/72 for i in range(1,73)]

In [ ]:
classification_report(dataset["Class"].loc[test_set], preds)

In [ ]:
# make a data frame containing the counts of light curve segments in each of the Gaussian components, for each observation
obs_component_counts_df_122 = pd.DataFrame(np.zeros((len(ObID_GaussComps_dict_122),len(np.unique(shape_moments_GMM122_labels)))), index=np.unique(seg_ObIDs), columns=list(range(122)), dtype=int)

In [ ]:
# populate the data frame
for ObID, GaussComps in ObID_GaussComps_dict_122.items():
    for comp_id, comp_count in np.array(np.unique(GaussComps, return_counts=True)).T:
        obs_component_counts_df_122.loc[ObID][comp_id] = comp_count
obs_component_counts_df_122

In [ ]:
from sklearn.svm import SVC
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
#(y_true, y_pred, *, beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')

# import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (3.32, 3.32)#(6.97, 6.97)
plt.rcParams.update({'font.size': 6})

experiment_list = [

    [obs_component_counts_df_122, "Classification with proportional make up of observations in terms of 114 Gaussian mixture components"],

]

reports = []
# for test_set, train_set in zip(test_sets, train_sets):
n=1#3
test_set=test_sets[n]
train_set=train_sets[n]
for dataset, title in experiment_list:
    SVC_clf = RandomForestClassifier(random_state=0, class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])
    if title[-8:] == "balanced":
        SVC_clf = SVC(gamma="auto", class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])

    preds = SVC_clf.predict(dataset.loc[test_set].iloc[:,:122])
#         reports.append(classification_report(dataset["Class"].loc[test_set], preds))
    reports.append(( precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="weighted"),precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="macro"), accuracy_score(dataset["Class"].loc[test_set], preds)))
#             precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="macro"), precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="micro")))
#     print("{}".format(title))
    print(classification_report(dataset["Class"].loc[test_set], preds))



    disp = plot_confusion_matrix(SVC_clf, dataset.loc[test_set].iloc[:,:122], dataset["Class"].loc[test_set],
                             cmap=plt.cm.Blues,
                             normalize=None)
#     disp.ax_.set_title("{}".format(title))
    disp.ax_.set_xlabel("Predicted class")
    disp.ax_.set_ylabel("True class")
    disp.ax_.set_yticklabels([r"$\{}$".format(class_name) for class_name in np.unique(dataset["Class"])[1:]])
    disp.ax_.set_xticklabels([r"$\{}$".format(class_name) for class_name in np.unique(dataset["Class"])[1:]])
    plt.savefig('figures/classification_matrix.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

    plt.show()


In [ ]:
misclas = np.where(dataset["Class"].loc[test_set] != preds)[0]

In [ ]:
np.where(ids == ob_id)

In [ ]:
plt.rcParams['figure.figsize'] = (13.32, 3.32)
for m in misclas:
    ob_id = dataset.loc[test_set].index.values[m]
    ob_ind = np.where(np.array(ids) == ob_id)[0][0]
    plt.plot(lcs[ob_ind][0], lcs[ob_ind][1])
    print("true: ", ob_state[ob_id], "predicted: ", preds[m])
    plt.show()

In [ ]:
[ob_state[x] for x in ["40703-01-30-00", "40703-01-30-01", "40703-01-30-02", "40703-01-14-00"]]

In [ ]:
inv_ob_state["kappa"]

In [ ]:
np.where(np.array(ids) == '40703-01-12-00')[0][0]

In [ ]:
plt.rcParams['figure.figsize'] = (130.32, 3.32)
for m in ["40703-01-30-00", "40703-01-30-01", "40703-01-30-02", "40703-01-14-00", "40703-01-12-00", "40703-01-26-00"]:
    ob_ind = np.where(np.array(ids) == m)[0][0]
    plt.plot(lcs[ob_ind][0], lcs[ob_ind][1])
    print("true: ", ob_state[m])
    plt.show()

In [ ]:
[r"$\{}$".format(class_name) for class_name in np.unique(dataset["Class"])[1:]]

# Classification without bad kappa/omega 

In [ ]:
compar_results = np.zeros((2,1))

# inverse the ob_state dictionary, so that inv_ob_state contains {"state name" : [list of observation IDs], ...}
inv_ob_state = {} 
for k, v in ob_state.items():
    if k in ["40703-01-30-00", "40703-01-30-01", "40703-01-30-02", "40703-01-14-00"]:
        continue
    inv_ob_state[v] = inv_ob_state.get(v, [])
    inv_ob_state[v].append(k)

test_sets = []
train_sets = []
# split the observations into test and training sets
for test_iter in range(10):
    print(test_iter)
    test_set = []
    train_set = []
    
    # split is stratified, so done separately for each class
    for class_name in ["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi", "eta", "omega"]:

        class_obs_all = inv_ob_state[class_name] # all labeled observation IDs of this class
        # exclude observations which did not produce any light curve segments
        class_obs = []
        for ob in class_obs_all:
            if ob in seg_ObIDs: # seg_ObIDs contains observation ID for each of the 468202 light curve segments
                class_obs.append(ob)
        
        # pick 1/3 of observations for the test set (previously replace=True was set!!!!)
        test_obs = np.random.choice(class_obs, size=int(np.ceil(len(class_obs)/3)), replace=False) 

        if len(test_obs) == 0:
            print(class_name)
        
        # use the remaining observations as training set
        train_obs = []
        for ob in class_obs:
            if ob not in test_obs:
                train_obs.append(ob)
        test_set.append(test_obs)
        train_set.append(train_obs)

    test_set=np.hstack(test_set)
    train_set=np.hstack(train_set)
    test_sets.append(test_set)
    train_sets.append(train_set)
    clear_output(wait=1)

In [ ]:
inv_ob_state["kappa"]

In [ ]:
train_set.shape

In [ ]:
pseudo_CLR_transformed

In [ ]:
obs_component_counts_df_122

In [ ]:
print(obs_component_counts_df_122["Class"].values[:100])

In [ ]:
pseudo_CLR_transformed["Class"] = "Unknown"
for k,v in ob_state.items():
    if str(k) in obs_component_counts_df_122.index.values:
        pseudo_CLR_transformed.loc[str(k), "Class"] = v

In [ ]:
obs_component_counts_df_122.iloc[:,:122].div(np.sum(obs_component_counts_df_122.iloc[:,:122], axis=1), axis="rows")
obs_component_counts_df_122["Class"] = "Unknown"

In [ ]:
for k,v in ob_state.items():
    if str(k) in obs_component_counts_df_122.index.values:
        obs_component_counts_df_122.loc[str(k), "Class"] = v

In [ ]:
obs_component_counts_df_122

In [ ]:
pseudo_CLR_transformed

In [ ]:
biclen(list(ob_state.items()))

In [ ]:
148+72

In [ ]:
len(test_set)

In [ ]:
obs_component_counts_df_122.loc[test_set].Class.values

In [ ]:
inv_ob_state

In [ ]:
for t in test_set:
    print(ob_state[t])

In [ ]:
obs_component_counts_df_122.iloc[:,:122] = obs_component_counts_df_122.iloc[:,:122].div(np.sum(obs_component_counts_df_122.iloc[:,:122], axis=1), axis="rows")

In [ ]:
from sklearn.svm import SVC
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
#(y_true, y_pred, *, beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')

# import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (6.97, 6.97)
plt.rcParams.update({'font.size': 6})

experiment_list = [

    [obs_component_counts_df_122, "Classification with proportional make up of observations in terms of 114 Gaussian mixture components"],

]
n=0
reports = []
for test_set, train_set in zip(test_sets, train_sets):
    print(n)
    n+=1
    clear_output(wait=True)
    for dataset, title in experiment_list:
        
        SVC_clf = RandomForestClassifier(random_state=0, class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])
        if title[-8:] == "balanced":
            SVC_clf = SVC(gamma="auto", class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])

        preds = SVC_clf.predict(dataset.loc[test_set].iloc[:,:122])
#         reports.append(classification_report(dataset["Class"].loc[test_set], preds))
        reports.append(( precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="weighted"),precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="macro"), accuracy_score(dataset["Class"].loc[test_set], preds)))
#             precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="macro"), precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="micro")))
#         print("{}".format(title))
#         print(classification_report(dataset["Class"].loc[test_set], preds))



#         disp = plot_confusion_matrix(SVC_clf, dataset.loc[test_set].iloc[:,:122], dataset["Class"].loc[test_set],
#                                  cmap=plt.cm.Blues,
#                                  normalize=None)
    #     disp.ax_.set_title("{}".format(title))
#         disp.ax_.set_xlabel("Predicted class")
#         disp.ax_.set_ylabel("True class")
    #     plt.savefig('figures/classification_matrix.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

    #     plt.show()
    
    

In [ ]:
for tt in test_sets:
    print(len(tt))

In [ ]:
dataset["Class"].loc[test_set]

In [ ]:
precision_recall_fscore_support(np.concatenate((np.ones(26), np.zeros(72-26))), np.ones(72),zero_division=0, average="weighted")

In [ ]:
print(np.mean(f1s_weighted), np.median(f1s_weighted), np.std(f1s_weighted))

In [ ]:
print(np.mean(accuracies), np.median(accuracies), np.std(accuracies))

In [ ]:
accuracies = []
f1s_weighted = []
f1s_average = []
for report in reports:# weighted(precision, recall, f1), accuracy
    accuracies.append(report[2])
    f1s_weighted.append(report[0][2])
    f1s_average.append(report[1][2])

In [ ]:
plt.rcParams['figure.figsize'] = (3.32, 3.32)
plt.rcParams.update({'font.size': 6})

# plt.hist(f1s_average, bins=15, label="average F1")
plt.hist(f1s_weighted, bins=15, label="weighted F1", alpha=0.7, color="blue")
plt.hist(accuracies, bins=15, label="accuracy", alpha=0.8, color="orange")
plt.legend()
# plt.xlim((0.65,0.95))
axes = plt.gca()
axes.tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
axes.tick_params(axis="y", which="major", length=2, width=0.75, labelsize=6, direction="in")
plt.xlabel("Test score")
plt.ylabel("No. classification experiments")

plt.axvline(0.19160997732426302, c="blue")

plt.axvline(0.3611111111111111, c="orange")


plt.savefig('figures/classification_histogram_lines.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
[i/72 for i in range(1,73)]

In [ ]:
classification_report(dataset["Class"].loc[test_set], preds)

In [ ]:
# make a data frame containing the counts of light curve segments in each of the Gaussian components, for each observation
obs_component_counts_df_122 = pd.DataFrame(np.zeros((len(ObID_GaussComps_dict_122),len(np.unique(shape_moments_GMM122_labels)))), index=np.unique(seg_ObIDs), columns=list(range(122)), dtype=int)

In [ ]:
# populate the data frame
for ObID, GaussComps in ObID_GaussComps_dict_122.items():
    for comp_id, comp_count in np.array(np.unique(GaussComps, return_counts=True)).T:
        obs_component_counts_df_122.loc[ObID][comp_id] = comp_count
obs_component_counts_df_122

In [ ]:
from sklearn.svm import SVC
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
#(y_true, y_pred, *, beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')

# import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (3.32, 3.32)#(6.97, 6.97)
plt.rcParams.update({'font.size': 6})

experiment_list = [

    [obs_component_counts_df_122, "Classification with proportional make up of observations in terms of 114 Gaussian mixture components"],

]

reports = []
# for test_set, train_set in zip(test_sets, train_sets):
n=1#3
test_set=test_sets[n]
train_set=train_sets[n]
for dataset, title in experiment_list:
    SVC_clf = RandomForestClassifier(random_state=0, class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])
    if title[-8:] == "balanced":
        SVC_clf = SVC(gamma="auto", class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], dataset["Class"].loc[train_set])

    preds = SVC_clf.predict(dataset.loc[test_set].iloc[:,:122])
#         reports.append(classification_report(dataset["Class"].loc[test_set], preds))
    reports.append(( precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="weighted"),precision_recall_fscore_support(dataset["Class"].loc[test_set], preds,zero_division=0, average="macro"), accuracy_score(dataset["Class"].loc[test_set], preds)))
#             precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="macro"), precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="micro")))
#     print("{}".format(title))
    print(classification_report(dataset["Class"].loc[test_set], preds))



    disp = plot_confusion_matrix(SVC_clf, dataset.loc[test_set].iloc[:,:122], dataset["Class"].loc[test_set],
                             cmap=plt.cm.Blues,
                             normalize=None)
#     disp.ax_.set_title("{}".format(title))
    disp.ax_.set_xlabel("Predicted class")
    disp.ax_.set_ylabel("True class")
    disp.ax_.set_yticklabels([r"$\{}$".format(class_name) for class_name in np.unique(dataset["Class"])[1:]])
    disp.ax_.set_xticklabels([r"$\{}$".format(class_name) for class_name in np.unique(dataset["Class"])[1:]])
    plt.savefig('figures/classification_matrix.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

    plt.show()


In [ ]:
for ob in inv_ob_state["nu"]:
    if ob in seg_ObIDs:
        print(ob)

# Classification with four chi sub classes

In [ ]:
# load observation classifications from Huppenkothen 2017
clean_belloni = open('{}/1915Belloniclass_updated.dat'.format(data_dir))
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
#     if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state

In [ ]:
ob_state

In [ ]:
compar_results = np.zeros((2,1))

# inverse the ob_state dictionary, so that inv_ob_state contains {"state name" : [list of observation IDs], ...}
inv_ob_state = {} 
for k, v in ob_state.items():
    inv_ob_state[v] = inv_ob_state.get(v, [])
    inv_ob_state[v].append(k)

test_sets = []
train_sets = []
# split the observations into test and training sets
for test_iter in range(1000):
    test_set = []
    train_set = []
    
    # split is stratified, so done separately for each class
    for class_name in ["alpha", "beta", "gamma", "delta", "theta", "kappa", "lambda", "mu", "nu", "rho", "phi", "chi1","chi2","chi3","chi4", "eta", "omega"]:

        class_obs_all = inv_ob_state[class_name] # all labeled observation IDs of this class
        # exclude observations which did not produce any light curve segments
        class_obs = []
        for ob in class_obs_all:
            if ob in seg_ObIDs: # seg_ObIDs contains observation ID for each of the 468202 light curve segments
                class_obs.append(ob)
        
        # pick 1/3 of observations for the test set (previously replace=True was set!!!!)
        test_obs = np.random.choice(class_obs, size=int(np.ceil(len(class_obs)/3)), replace=False) 

        if len(test_obs) == 0:
            print(class_name)
        
        # use the remaining observations as training set
        train_obs = []
        for ob in class_obs:
            if ob not in test_obs:
                train_obs.append(ob)
        test_set.append(test_obs)
        train_set.append(train_obs)

    test_set=np.hstack(test_set)
    train_set=np.hstack(train_set)
    test_sets.append(test_set)
    train_sets.append(train_set)


In [ ]:
inv_ob_state.keys()

In [ ]:
test_sets[0].shape

In [ ]:
train_sets[0].shape

In [ ]:
pseudo_CLR_transformed

In [ ]:
obs_component_counts_df_122

In [ ]:
print(obs_component_counts_df_122["Class"].values[:100])

In [ ]:
pseudo_CLR_transformed["Class"] = "Unknown"
for k,v in ob_state.items():
    if str(k) in obs_component_counts_df_122.index.values:
        pseudo_CLR_transformed.loc[str(k), "Class"] = v

In [ ]:
obs_component_counts_df_122.iloc[:,:122].div(np.sum(obs_component_counts_df_122.iloc[:,:122], axis=1), axis="rows")
obs_component_counts_df_122["Class"] = "Unknown"

In [ ]:
for k,v in ob_state.items():
    if str(k) in obs_component_counts_df_122.index.values:
        obs_component_counts_df_122.loc[str(k), "Class"] = v

In [ ]:
obs_component_counts_df_122

In [ ]:
pseudo_CLR_transformed

In [ ]:
len(list(ob_state.items()))

In [ ]:
148+72

In [ ]:
len(test_set)

In [ ]:
[ob_state[x] for x in obs_component_counts_df_122.loc[test_set].index.values]

In [ ]:
inv_ob_state

In [ ]:
for t in test_set:
    print(ob_state[t])

In [ ]:
obs_component_counts_df_122.iloc[:,:122] = obs_component_counts_df_122.iloc[:,:122].div(np.sum(obs_component_counts_df_122.iloc[:,:122], axis=1), axis="rows")

In [ ]:
from sklearn.svm import SVC
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
#(y_true, y_pred, *, beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')

# import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (6.97, 6.97)
plt.rcParams.update({'font.size': 6})

experiment_list = [

    [obs_component_counts_df_122, "Classification with proportional make up of observations in terms of 114 Gaussian mixture components"],

]


reports = []
for test_set, train_set in zip(test_sets, train_sets):

    for dataset, title in experiment_list:
        SVC_clf = RandomForestClassifier(random_state=0, class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], [ob_state[x] for x in train_set])
        if title[-8:] == "balanced":
            SVC_clf = SVC(gamma="auto", class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], [ob_state[x] for x in train_set])

        preds = SVC_clf.predict(dataset.loc[test_set].iloc[:,:122])
#         reports.append(classification_report(dataset["Class"].loc[test_set], preds))
        reports.append(( precision_recall_fscore_support([ob_state[x] for x in test_set], preds,zero_division=0, average="weighted"),precision_recall_fscore_support([ob_state[x] for x in test_set], preds,zero_division=0, average="macro"), accuracy_score([ob_state[x] for x in test_set], preds)))
#             precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="macro"), precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="micro")))
#         print("{}".format(title))
#         print(classification_report(dataset["Class"].loc[test_set], preds))



#         disp = plot_confusion_matrix(SVC_clf, dataset.loc[test_set].iloc[:,:122], dataset["Class"].loc[test_set],
#                                  cmap=plt.cm.Blues,
#                                  normalize=None)
    #     disp.ax_.set_title("{}".format(title))
#         disp.ax_.set_xlabel("Predicted class")
#         disp.ax_.set_ylabel("True class")
    #     plt.savefig('figures/classification_matrix.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

    #     plt.show()
    
    

In [ ]:
accuracies = []
f1s_weighted = []
f1s_average = []
for report in reports:# weighted(precision, recall, f1), accuracy
    accuracies.append(report[2])
    f1s_weighted.append(report[0][2])
    f1s_average.append(report[1][2])

In [ ]:
plt.rcParams['figure.figsize'] = (3.32, 3.32)
plt.rcParams.update({'font.size': 6})

plt.hist(f1s_average, bins=15, label="average F1")
plt.hist(f1s_weighted, bins=15, label="weighted F1", alpha=0.7)
plt.hist(accuracies, bins=15, label="accuracy", alpha=0.7)
plt.legend()
# plt.xlim((0.65,0.95))
axes = plt.gca()
axes.tick_params(axis="x", which="major", length=2, width=0.75, labelsize=6, direction="in")
axes.tick_params(axis="y", which="major", length=2, width=0.75, labelsize=6, direction="in")
plt.xlabel("Test score")
plt.ylabel("No. classification experiments")
plt.title("Classification with 4 chi subclasses")


plt.savefig('figures/classification_histogram_chi_subclasses.png', dpi=300, bbox_inches = 'tight',pad_inches = 0.01)

plt.show()

In [ ]:
print(np.mean(f1s_weighted), np.median(f1s_weighted))

In [ ]:
print(np.mean(accuracies), np.median(accuracies))

In [ ]:
[i/72 for i in range(1,73)]

In [ ]:
classification_report(dataset["Class"].loc[test_set], preds)

In [ ]:
# make a data frame containing the counts of light curve segments in each of the Gaussian components, for each observation
obs_component_counts_df_122 = pd.DataFrame(np.zeros((len(ObID_GaussComps_dict_122),len(np.unique(shape_moments_GMM122_labels)))), index=np.unique(seg_ObIDs), columns=list(range(122)), dtype=int)

In [ ]:
# populate the data frame
for ObID, GaussComps in ObID_GaussComps_dict_122.items():
    for comp_id, comp_count in np.array(np.unique(GaussComps, return_counts=True)).T:
        obs_component_counts_df_122.loc[ObID][comp_id] = comp_count
obs_component_counts_df_122

In [ ]:
from sklearn.svm import SVC
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
#(y_true, y_pred, *, beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')

# import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (3.32, 3.32)#(6.97, 6.97)
plt.rcParams.update({'font.size': 6})

experiment_list = [

    [obs_component_counts_df_122, "Classification with proportional make up of observations in terms of 114 Gaussian mixture components"],

]

reports = []
# for test_set, train_set in zip(test_sets, train_sets):
n=1#3
test_set=test_sets[n]
train_set=train_sets[n]
for dataset, title in experiment_list:
    SVC_clf = RandomForestClassifier(random_state=0, class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], [ob_state[x] for x in train_set])
    if title[-8:] == "balanced":
        SVC_clf = SVC(gamma="auto", class_weight="balanced").fit(dataset.loc[train_set].iloc[:,:122], [ob_state[x] for x in train_set])

    preds = SVC_clf.predict(dataset.loc[test_set].iloc[:,:122])
#         reports.append(classification_report(dataset["Class"].loc[test_set], preds))
    reports.append(( precision_recall_fscore_support([ob_state[x] for x in test_set], preds,zero_division=0, average="weighted"),precision_recall_fscore_support([ob_state[x] for x in test_set], preds,zero_division=0, average="macro"), accuracy_score([ob_state[x] for x in test_set], preds)))
#             precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="macro"), precision_recall_fscore_support(dataset["Class"].loc[test_set], preds, average="micro")))
#     print("{}".format(title))
    print(classification_report([ob_state[x] for x in test_set], preds))



    disp = plot_confusion_matrix(SVC_clf, dataset.loc[test_set].iloc[:,:122], [ob_state[x] for x in test_set],
                             cmap=plt.cm.Blues,
                             normalize=None)
# #     disp.ax_.set_title("{}".format(title))
#     disp.ax_.set_xlabel("Predicted class")
#     disp.ax_.set_ylabel("True class")
#     disp.ax_.set_yticklabels([r"$\{}$".format(class_name) for class_name in np.unique([ob_state[x] for x in test_set])[1:]])
#     disp.ax_.set_xticklabels([r"$\{}$".format(class_name) for class_name in np.unique([ob_state[x] for x in test_set])[1:]])
# #     plt.savefig('figures/classification_matrix.png', dpi=300, bbox_inches = 'tight',pad_inches = 0)

    plt.show()
